In [1]:
import numpy as np
import xarray as xr
import os
import glob
from pathlib import Path 
import pathlib
from pprint import pprint
import matplotlib.pyplot as plt
import time
import requests
import s3fs
from os.path import dirname, join
import datetime
import myDate
import netCDF4
import gc

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
warnings.filterwarnings('ignore')

In [2]:
alongtrack_file_dir = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output/')
alongtrack_file_dir_randomnoise = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise/')
alongtrack_file_dir_missingdata = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_missingdata/')
alongtrack_file_dir_orbiterror_1cm = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_1cm/')
alongtrack_file_dir_3errors_1cm = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_1cm/')
alongtrack_file_dir_orbiterror_2cm = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_2cm/')
alongtrack_file_dir_3errors_2cm = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_2cm/')
alongtrack_file_dir_orbiterror_3cm = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_3cm/')
alongtrack_file_dir_3errors_3cm = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_3cm/')
grids_file_dir = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output/')
grids_file_dir_randomnoise = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/')
grids_file_dir_missingdata = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/')
grids_file_dir_orbiterror_1cm = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_1cm/')
grids_file_dir_3errors_1cm = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_1cm/')
grids_file_dir_orbiterror_2cm = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_2cm/')
grids_file_dir_3errors_2cm = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_2cm/')
grids_file_dir_orbiterror_3cm = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_3cm/')
grids_file_dir_3errors_3cm = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_3cm/')
grids_file_dir_beckley = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_Beckley_dailyfiles/')
beckley_dailyfile_dir = Path('/home/jovyan/Data/ECCO_GMSL/Beckley_dailyfiles/')
beckley_file_dir = Path('/home/jovyan/Data/ECCO_GMSL/Beckley/')
grids_file_dir_JPL = Path('/home/jovyan/Data/ECCO_GMSL/JPL_measures/')
main_dir = Path('/home/jovyan/ECCO_GMSL/')
output_dir = Path('/home/jovyan/Data/ECCO_GMSL/GMSL/')

# SIC dir
sic_dir=Path('/home/jovyan/Data/ECCO_GMSL/SIC_ECCO_DAILY_V4R4B/')

ecco_mean = xr.open_dataset(str(Path('/home/jovyan/Data/ECCO_GMSL')) + '/ECCO_V4r4_mean_SSH.nc')
ecco_mean_all = xr.open_dataset(str(Path('/home/jovyan/Data/ECCO_GMSL')) + '/ECCO_V4r4_mean_SSH_alllat.nc')
# ecco_mean_nosicapplied = xr.open_dataset(str(Path('/home/jovyan/Data/ECCO_GMSL')) + '/ECCO_V4r4_mean_SSH_nosicapplied.nc')
# ecco_mean_all_nosicapplied = xr.open_dataset(str(Path('/home/jovyan/Data/ECCO_GMSL')) + '/ECCO_V4r4_mean_SSH_alllatnosicapplied.nc')

# Compute Beckley along track GMSL
### Compute parameters

In [18]:
Vs = 7.2 #satellite along track velocity in km/s
Ve = 0.46 #velocity of earth's rotation at the equator in km/s
incidence = np.deg2rad(66.04) #satellite inclination
sig = 1 #uncertainty
alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(0)))
gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(0)) / (Vs*np.cos(alpha)) ))
beta0=np.pi/2 - gamma

#### Compute GMSL 10-day time series

In [19]:
beckley_alongtrack_file_dir = list(beckley_file_dir.glob('Merged*Cycle*nc'))
beckley_alongtrack_file_dir=np.sort(beckley_alongtrack_file_dir)
beckley_alongtrack_file_dir

array([PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley/Merged_TOPEX_Jason_OSTM_Jason-3_Cycle_0001.V5_1.nc'),
       PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley/Merged_TOPEX_Jason_OSTM_Jason-3_Cycle_0002.V5_1.nc'),
       PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley/Merged_TOPEX_Jason_OSTM_Jason-3_Cycle_0003.V5_1.nc'),
       ...,
       PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley/Merged_TOPEX_Jason_OSTM_Jason-3_Cycle_1057.V5_1.nc'),
       PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley/Merged_TOPEX_Jason_OSTM_Jason-3_Cycle_1058.V5_1.nc'),
       PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley/Merged_TOPEX_Jason_OSTM_Jason-3_Cycle_1059.V5_1.nc')],
      dtype=object)

In [20]:
beckley_gmsl_tmp=[]
for i in np.arange(0,len(beckley_alongtrack_file_dir)):
    alongtrack = xr.open_mfdataset(
            paths=beckley_alongtrack_file_dir[i],
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.ssha*10**(-3)
    # SSH_at_xy_ano=alongtrack.ssha-ref_mean
    alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    beta = np.pi/2 - gamma
    w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w=w.where(np.isfinite(SSH_at_xy_ano))
    beckley_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
beckley_gmsl = xr.concat(beckley_gmsl_tmp, dim='time')

In [21]:
alongtrack

<xarray.Dataset>
Dimensions:            (i: 1, N_Records: 602812)
Coordinates:
    lat                (N_Records) float32 dask.array<chunksize=(602812,), meta=np.ndarray>
    lon                (N_Records) float32 dask.array<chunksize=(602812,), meta=np.ndarray>
    time               (N_Records) datetime64[ns] dask.array<chunksize=(602812,), meta=np.ndarray>
Dimensions without coordinates: i, N_Records
Data variables:
    index              (i, N_Records) float32 dask.array<chunksize=(1, 602812), meta=np.ndarray>
    reference_orbit    (i, N_Records) float32 dask.array<chunksize=(1, 602812), meta=np.ndarray>
    mssh               (i, N_Records) float64 dask.array<chunksize=(1, 602812), meta=np.ndarray>
    Distance_to_coast  (i, N_Records) float64 dask.array<chunksize=(1, 602812), meta=np.ndarray>
    Surface_Type       (i, N_Records) float32 dask.array<chunksize=(1, 602812), meta=np.ndarray>
    Bathymetry         (i, N_Records) float32 dask.array<chunksize=(1, 602812), meta=np.ndarray>
    flag               (i, N_Records) float32 dask.array<chunksize=(1, 602812), meta=np.ndarray>
    ssha               (i, N_Records) float64 dask.array<chunksize=(1, 602812), meta=np.ndarray>
    mss_modified       (i, N_Records) float64 dask.array<chunksize=(1, 602812), meta=np.ndarray>
    old_mssh           (i, N_Records) float64 dask.array<chunksize=(1, 602812), meta=np.ndarray>
    old_ssha           (i, N_Records) float32 dask.array<chunksize=(1, 602812), meta=np.ndarray>
Attributes: (12/46)
    Conventions:                   CF-1.7, ACDD-1.3
    title:                         Integrated Multi-Mission Ocean Altimeter D...
    summary:                       Geo-registered along-track sea surface hei...
    source:                        TOPEX/Poseidon MGDR_B: Benada, J.R. 1997. ...
    processing_level:              2
    featureType:                   trajectory
    ...                            ...
    geospatial_vertical_units:     mm
    geospatial_vertical_positive:  up
    geospatial_lat_resolution:     0.05
    geospatial_lon_resolution:     2.83
    geospatial_vertical_min:       -8666
    geospatial_vertical_max:       8738

In [23]:
beckley_gmsl.name = 'gmsl_beckley_alongtrack'
beckley_gmsl.attrs['units'] = 'm'
beckley_gmsl.attrs['summary'] = '10-day GMSL time series computed on the alongtrack GSFC data'#, referenced to a 1996-2016 global mean SSH'
beckley_gmsl.time.attrs['long_name'] = 'mid day of the 10-day average'
#we use the Beckley time series for the time
gmsl_beckley = xr.open_dataset(str(output_dir) + '/GMSL_Beckley_v51.nc')
beckley_gmsl['time']=gmsl_beckley.time[0:len(beckley_gmsl)]
beckley_gmsl

<xarray.DataArray 'gmsl_beckley_alongtrack' (time: 1058)>
dask.array<concatenate, shape=(1058,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1993-01-05 1993-01-15 ... 2021-09-25
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack GSFC data

In [24]:
fname = output_dir / ('GMSL_Beckley.nc')
beckley_gmsl.to_netcdf(fname)

# Compute Beckley's daily files along track GMSL (Beckley's style)

In [25]:
beckley_alongtrack_dailyfile_dir = list(beckley_dailyfile_dir.glob('MERGED*h5'))
beckley_alongtrack_dailyfile_dir=np.sort(beckley_alongtrack_dailyfile_dir)
beckley_alongtrack_dailyfile_dir

array([PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley_dailyfiles/MERGED_ALT-alt_ssh19920925.h5'),
       PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley_dailyfiles/MERGED_ALT-alt_ssh19920926.h5'),
       PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley_dailyfiles/MERGED_ALT-alt_ssh19920927.h5'),
       ...,
       PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley_dailyfiles/MERGED_ALT-alt_ssh20171229.h5'),
       PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley_dailyfiles/MERGED_ALT-alt_ssh20171230.h5'),
       PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley_dailyfiles/MERGED_ALT-alt_ssh20171231.h5')],
      dtype=object)

In [26]:
beckley_gmsl_tmp=[]
time_gmsl=[]

start_date = np.datetime64('1950-01-01')
time_fiction=range(myDate.date2jj(1992,9,23),myDate.date2jj(2017,12,31)+11)
for i in np.arange(0,len(time_fiction)-10,10):
    lat=[]
    lon=[]
    ssh=[]
    ssh_smoothed=[]
    time=[]
    for j in range(0,10):
        [year,month,day]=myDate.jj2date(time_fiction[i+j])
        try:
            ECCO_alongtrack_file_cycle=list(beckley_dailyfile_dir.glob('MERGED_ALT-alt_ssh'+str(year)+str(month).zfill(2)+str(day).zfill(2)+'*h5'))[0]
            ncf = netCDF4.Dataset(ECCO_alongtrack_file_cycle, diskless=True, persist=False)
            nch = ncf.groups.get('data')
            xds = xr.open_dataset(xr.backends.NetCDF4DataStore(nch))
            lat=np.hstack((lat,xds.lats))
            lon=np.hstack((lon,xds.lons))
            ssh=np.hstack((ssh,xds.ssh))
            ssh_smoothed=np.hstack((ssh_smoothed,xds.ssh_smoothed))
            time=np.hstack((time,xds.time))
        except:
            print('no file on: '+str(year)+str(month).zfill(2)+str(day).zfill(2))
    try:
        alongtrack = {
            'ssh': xr.DataArray(ssh, dims='time', coords={'time': time}),
            'lat': xr.DataArray(lat, dims='time', coords={'time': time}),
            'lon': xr.DataArray(lon, dims='time', coords={'time': time})
        }
        alongtrack = xr.Dataset(alongtrack)
        alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
        SSH_at_xy_ano=alongtrack.ssh
        alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
        gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
        beta = np.pi/2 - gamma
        w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
        w=w.where(np.isfinite(SSH_at_xy_ano))
        time_gmsl = np.hstack((time_gmsl,time_fiction[i]))
        beckley_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
    except:
        time_gmsl = np.hstack((time_gmsl,time_fiction[i]))
        beckley_gmsl_tmp.append(np.nan)

no file on: 19920923
no file on: 19920924
no file on: 19930723
no file on: 19930724
no file on: 19930725
no file on: 19940317
no file on: 19951127
no file on: 19951128
no file on: 19951129
no file on: 19951130
no file on: 19951201
no file on: 19951202
no file on: 19951203
no file on: 19951204
no file on: 19951205
no file on: 19951206
no file on: 19960121
no file on: 19960122
no file on: 19970605
no file on: 19970606
no file on: 19990210
no file on: 19990831
no file on: 19990901
no file on: 19990902
no file on: 19990903
no file on: 20000403
no file on: 20000404
no file on: 20000405
no file on: 20031120
no file on: 20031121
no file on: 20031122
no file on: 20031123
no file on: 20031124
no file on: 20031125
no file on: 20031126
no file on: 20031127
no file on: 20040216
no file on: 20040217
no file on: 20040218
no file on: 20040219
no file on: 20040220
no file on: 20040404
no file on: 20050921
no file on: 20050922
no file on: 20050923
no file on: 20050924
no file on: 20050925
no file on: 2

In [27]:
beckley_gmsl = xr.concat(beckley_gmsl_tmp, dim='time')
beckley_gmsl

<xarray.DataArray (time: 924)>
array([ 1.62416671e-02,  1.08792421e-02, -6.94041271e-05, -5.20266182e-03,
       -5.95520338e-03, -9.17995665e-03, -4.22096797e-03, -1.35513535e-02,
       -1.62346569e-02, -1.71985476e-02, -1.73986995e-02, -2.02089226e-02,
       -2.35727351e-02, -2.08680434e-02, -1.73705464e-02, -1.73802980e-02,
       -1.66112562e-02, -1.71774281e-02, -1.94314508e-02, -1.86880137e-02,
       -2.02587340e-02, -1.92052959e-02, -2.09768841e-02, -2.19199938e-02,
       -2.41426726e-02, -2.29135279e-02, -2.00988133e-02, -1.88881783e-02,
       -1.78331529e-02, -1.77239311e-02, -1.39944070e-02, -1.87751412e-02,
       -1.51715762e-02, -1.22013413e-02, -1.24087930e-02, -1.34349120e-02,
       -1.25428515e-02, -1.04827824e-02, -6.97279011e-03, -1.01272638e-02,
       -1.28704715e-02, -1.39213668e-02, -1.49672597e-02, -1.55727648e-02,
       -1.12511559e-02, -9.99076250e-03, -1.19253552e-02, -1.43832315e-02,
       -1.04181292e-02, -1.11450106e-02, -1.71232663e-02, -1.79293668e-02,
       -1.63043803e-02, -1.40130505e-02, -1.24936015e-02, -1.37782371e-02,
       -1.50956429e-02, -1.55335963e-02, -1.87429085e-02, -1.98458290e-02,
       -1.98611953e-02, -1.84183504e-02, -1.93300451e-02, -1.88131919e-02,
       -2.27946635e-02, -1.72553102e-02, -1.65351240e-02, -1.44346946e-02,
       -1.32452493e-02, -1.16875263e-02, -1.05657864e-02, -7.83623642e-03,
       -8.48177449e-03, -9.77769578e-03, -6.60340436e-03, -5.79864356e-03,
       -6.42639255e-03, -2.39978308e-03, -8.39972095e-03, -1.17846054e-02,
...
        6.09509939e-02,  5.86003110e-02,  6.02502811e-02,  5.86738714e-02,
        5.64023437e-02,  5.49857242e-02,  5.24395273e-02,  5.89632260e-02,
        5.98925366e-02,  5.57362837e-02,  5.85844220e-02,  5.57696732e-02,
        5.55364113e-02,  5.31332797e-02,  5.65575079e-02,  5.13532336e-02,
        5.29783428e-02,  4.95766067e-02,  4.81494817e-02,  4.62578922e-02,
        4.58806264e-02,  4.94362040e-02,  5.04278395e-02,  5.03489275e-02,
        5.15791630e-02,  5.17591932e-02,  5.01082254e-02,  5.08293667e-02,
        5.06669761e-02,  5.23988168e-02,  5.82603470e-02,  6.05917151e-02,
        5.81028550e-02,  5.62420105e-02,  5.72084851e-02,  6.13701551e-02,
        5.84578470e-02,  6.04801052e-02,  6.27236868e-02,  6.17747554e-02,
        5.84165435e-02,  5.73973630e-02,  5.62940925e-02,  5.66697547e-02,
        5.46751459e-02,  5.39874181e-02,  5.12272101e-02,  5.30635740e-02,
        5.28294307e-02,  5.27198240e-02,  5.29260241e-02,  5.16509013e-02,
        5.07984481e-02,  4.95862219e-02,  4.76610679e-02,  4.67475408e-02,
        4.71261814e-02,  4.77123769e-02,  4.95144978e-02,  5.12138821e-02,
        5.05865341e-02,  5.03690222e-02,  5.12023439e-02,  5.56782155e-02,
        5.67304812e-02,  5.91064877e-02,  5.97305310e-02,  6.26563122e-02,
        6.39190985e-02,  6.15428098e-02,  6.18044052e-02,  6.23371248e-02,
        6.39811449e-02,  6.49132245e-02,  6.58875547e-02,  6.56024180e-02,
        6.39646064e-02,  6.45208032e-02,  6.23270390e-02,  6.06392709e-02])
Dimensions without coordinates: time

In [28]:
beckley_gmsl.name = 'gmsl_beckley_daily_alongtrack'
beckley_gmsl.attrs['units'] = 'm'
beckley_gmsl.attrs['summary'] = '10-day GMSL time series computed on the daily file alongtrack GSFC data'#, referenced to a 1996-2016 global mean SSH'
beckley_gmsl.time.attrs['long_name'] = 'mid day of the 10-day average'
beckley_gmsl['time']= start_date + np.timedelta64(1, 'D') * (time_gmsl+4) 
beckley_gmsl

<xarray.DataArray 'gmsl_beckley_daily_alongtrack' (time: 924)>
array([ 1.62416671e-02,  1.08792421e-02, -6.94041271e-05, -5.20266182e-03,
       -5.95520338e-03, -9.17995665e-03, -4.22096797e-03, -1.35513535e-02,
       -1.62346569e-02, -1.71985476e-02, -1.73986995e-02, -2.02089226e-02,
       -2.35727351e-02, -2.08680434e-02, -1.73705464e-02, -1.73802980e-02,
       -1.66112562e-02, -1.71774281e-02, -1.94314508e-02, -1.86880137e-02,
       -2.02587340e-02, -1.92052959e-02, -2.09768841e-02, -2.19199938e-02,
       -2.41426726e-02, -2.29135279e-02, -2.00988133e-02, -1.88881783e-02,
       -1.78331529e-02, -1.77239311e-02, -1.39944070e-02, -1.87751412e-02,
       -1.51715762e-02, -1.22013413e-02, -1.24087930e-02, -1.34349120e-02,
       -1.25428515e-02, -1.04827824e-02, -6.97279011e-03, -1.01272638e-02,
       -1.28704715e-02, -1.39213668e-02, -1.49672597e-02, -1.55727648e-02,
       -1.12511559e-02, -9.99076250e-03, -1.19253552e-02, -1.43832315e-02,
       -1.04181292e-02, -1.11450106e-02, -1.71232663e-02, -1.79293668e-02,
       -1.63043803e-02, -1.40130505e-02, -1.24936015e-02, -1.37782371e-02,
       -1.50956429e-02, -1.55335963e-02, -1.87429085e-02, -1.98458290e-02,
       -1.98611953e-02, -1.84183504e-02, -1.93300451e-02, -1.88131919e-02,
       -2.27946635e-02, -1.72553102e-02, -1.65351240e-02, -1.44346946e-02,
       -1.32452493e-02, -1.16875263e-02, -1.05657864e-02, -7.83623642e-03,
       -8.48177449e-03, -9.77769578e-03, -6.60340436e-03, -5.79864356e-03,
       -6.42639255e-03, -2.39978308e-03, -8.39972095e-03, -1.17846054e-02,
...
        6.09509939e-02,  5.86003110e-02,  6.02502811e-02,  5.86738714e-02,
        5.64023437e-02,  5.49857242e-02,  5.24395273e-02,  5.89632260e-02,
        5.98925366e-02,  5.57362837e-02,  5.85844220e-02,  5.57696732e-02,
        5.55364113e-02,  5.31332797e-02,  5.65575079e-02,  5.13532336e-02,
        5.29783428e-02,  4.95766067e-02,  4.81494817e-02,  4.62578922e-02,
        4.58806264e-02,  4.94362040e-02,  5.04278395e-02,  5.03489275e-02,
        5.15791630e-02,  5.17591932e-02,  5.01082254e-02,  5.08293667e-02,
        5.06669761e-02,  5.23988168e-02,  5.82603470e-02,  6.05917151e-02,
        5.81028550e-02,  5.62420105e-02,  5.72084851e-02,  6.13701551e-02,
        5.84578470e-02,  6.04801052e-02,  6.27236868e-02,  6.17747554e-02,
        5.84165435e-02,  5.73973630e-02,  5.62940925e-02,  5.66697547e-02,
        5.46751459e-02,  5.39874181e-02,  5.12272101e-02,  5.30635740e-02,
        5.28294307e-02,  5.27198240e-02,  5.29260241e-02,  5.16509013e-02,
        5.07984481e-02,  4.95862219e-02,  4.76610679e-02,  4.67475408e-02,
        4.71261814e-02,  4.77123769e-02,  4.95144978e-02,  5.12138821e-02,
        5.05865341e-02,  5.03690222e-02,  5.12023439e-02,  5.56782155e-02,
        5.67304812e-02,  5.91064877e-02,  5.97305310e-02,  6.26563122e-02,
        6.39190985e-02,  6.15428098e-02,  6.18044052e-02,  6.23371248e-02,
        6.39811449e-02,  6.49132245e-02,  6.58875547e-02,  6.56024180e-02,
        6.39646064e-02,  6.45208032e-02,  6.23270390e-02,  6.06392709e-02])
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the daily file alongtrack G...

In [29]:
fname = output_dir / ('GMSL_Beckley_dailyfiles.nc')
beckley_gmsl.to_netcdf(fname)

# Compute ECCO along track GMSL (Beckley's style)

### No error
#### Compute the reference global mean SSH over 20 years 1996-2016

In [8]:
# ECCO_alongtrack_files=[]
# for year in range(1996,2017):
#     ECCO_alongtrack_files_tmp = list(alongtrack_file_dir.glob('*ECCO_V4r4_alongtrack_SSH_*'+str(year)+'*nc'))
#     ECCO_alongtrack_files.extend(ECCO_alongtrack_files_tmp)
# print(np.sort(ECCO_alongtrack_files))

In [9]:
# alongtrack = xr.open_mfdataset(
#         paths=ECCO_alongtrack_files,
#         combine='nested',
#         concat_dim='i',
#         decode_cf=True,
#         compat='override',
#         coords='minimal',
#         chunks={'time': 1}  
#     )  
# alongtrack

In [10]:
# ref_mean=alongtrack.SSH_at_xy.mean(skipna=True).values
# print(ref_mean)

#### Compute GMSL 10-day time series

In [30]:
ECCO_alongtrack_files = list(alongtrack_file_dir.glob('*ECCO_V4r4_alongtrack_SSH*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
print(ECCO_alongtrack_files)

[PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output/ECCO_V4r4_alongtrack_SSH_1992-09-23.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output/ECCO_V4r4_alongtrack_SSH_1992-09-24.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output/ECCO_V4r4_alongtrack_SSH_1992-09-25.nc')
 ...
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output/ECCO_V4r4_alongtrack_SSH_2017-12-29.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output/ECCO_V4r4_alongtrack_SSH_2017-12-30.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output/ECCO_V4r4_alongtrack_SSH_2017-12-31.nc')]


In [ ]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    beta = np.pi/2 - gamma
    w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
    
    # fig = plt.figure(figsize=(15,5))
    # cm = plt.get_cmap('gist_rainbow')
    # ax=fig.gca()
    # ax = plt.axes(projection=ccrs.Robinson( \
    #               central_longitude=-67, globe=None))
    # ax.gridlines()
    # ax.add_feature(cfeature.LAND)
    # ax.add_feature(cfeature.COASTLINE)
    # kk=12
    # p=ax.scatter(alongtrack.lon[::kk],\
    #                      alongtrack.lat[::kk], \
    #                      c=(SSH_at_xy_ano*w)[::kk], s=1,\
    #                      transform=ccrs.PlateCarree(),
    #                      vmin=-1,vmax=1, cmap='jet')
alongtrack_gmsl = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [ ]:
alongtrack_gmsl

In [ ]:
SSH_at_xy_ano

In [ ]:
alongtrack_gmsl=alongtrack_gmsl.drop('cycle_day')
alongtrack_gmsl=alongtrack_gmsl.drop('delta_days')
alongtrack_gmsl.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl.attrs['units'] = 'm'
alongtrack_gmsl.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl.time.attrs['long_name'] = 'mid day of the 10-day average'
alongtrack_gmsl['time']=alongtrack_gmsl.time+(4*24*60*60*10**(9))
alongtrack_gmsl

In [ ]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_noerror_nomean.nc')
alongtrack_gmsl.to_netcdf(fname)

### Random noise error
#### Compute the reference global mean SSH over 20 years 1996-2016

In [16]:
# ECCO_alongtrack_files=[]
# for year in range(1996,2017):
#     ECCO_alongtrack_files_tmp = list(alongtrack_file_dir_randomnoise.glob('*ECCO_V4r4_alongtrack_SSH_*'+str(year)+'*nc'))
#     ECCO_alongtrack_files.extend(ECCO_alongtrack_files_tmp)
# print(np.sort(ECCO_alongtrack_files))

In [17]:
# alongtrack = xr.open_mfdataset(
#         paths=ECCO_alongtrack_files,
#         combine='nested',
#         concat_dim='i',
#         decode_cf=True,
#         compat='override',
#         coords='minimal',
#         chunks={'time': 1}  
#     )  
# alongtrack

In [18]:
# ref_mean_randomnoise=alongtrack.SSH_at_xy.mean(skipna=True).values
# print(ref_mean_randomnoise)

#### Compute GMSL 10-day time series

In [19]:
ECCO_alongtrack_files = list(alongtrack_file_dir_randomnoise.glob('*ECCO_V4r4_alongtrack_SSH_*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
print(ECCO_alongtrack_files)

[PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise/ECCO_V4r4_alongtrack_SSH_1992-09-23_randomnoise.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise/ECCO_V4r4_alongtrack_SSH_1992-09-24_randomnoise.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise/ECCO_V4r4_alongtrack_SSH_1992-09-25_randomnoise.nc')
 ...
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise/ECCO_V4r4_alongtrack_SSH_2017-12-29_randomnoise.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise/ECCO_V4r4_alongtrack_SSH_2017-12-30_randomnoise.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise/ECCO_V4r4_alongtrack_SSH_2017-12-31_randomnoise.nc')]


In [20]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    beta = np.pi/2 - gamma
    w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
alongtrack_gmsl_randomnoise = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [21]:
alongtrack_gmsl_randomnoise=alongtrack_gmsl_randomnoise.drop('cycle_day')
alongtrack_gmsl_randomnoise=alongtrack_gmsl_randomnoise.drop('delta_days')
alongtrack_gmsl_randomnoise.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_randomnoise.attrs['units'] = 'm'
alongtrack_gmsl_randomnoise.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with random noise, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_randomnoise.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_randomnoise['time']=alongtrack_gmsl_randomnoise.time+(4*24*60*60*10**(9))
alongtrack_gmsl_randomnoise

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

In [22]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_randomnoise_nomean.nc')
alongtrack_gmsl_randomnoise.to_netcdf(fname)

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in arcsin
  return func(*(_execute_task(a, cache) for a in args))


### Missing data
#### Compute the reference global mean SSH over 20 years 1996-2016

In [23]:
# ECCO_alongtrack_files=[]
# for year in range(1996,2017):
#     ECCO_alongtrack_files_tmp = list(alongtrack_file_dir_missingdata.glob('*ECCO_V4r4_alongtrack_SSH_*'+str(year)+'*nc'))
#     ECCO_alongtrack_files.extend(ECCO_alongtrack_files_tmp)
# print(np.sort(ECCO_alongtrack_files))

In [24]:
# alongtrack = xr.open_mfdataset(
#         paths=ECCO_alongtrack_files,
#         combine='nested',
#         concat_dim='i',
#         decode_cf=True,
#         compat='override',
#         coords='minimal',
#         chunks={'time': 1}  
#     )  
# alongtrack

In [25]:
# ref_mean_missingdata=alongtrack.SSH_at_xy.mean(skipna=True).values
# print(ref_mean_missingdata)

#### Compute GMSL 10-day time series

In [26]:
ECCO_alongtrack_files = list(alongtrack_file_dir_missingdata.glob('*ECCO_V4r4_alongtrack_SSH*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
# print(ECCO_alongtrack_files)

In [27]:
alongtrack_gmsl_tmp_missingdata=[]
ind_vec=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano_missingdata=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    beta = np.pi/2 - gamma
    w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w=w.where(np.isfinite(SSH_at_xy_ano_missingdata))
    alongtrack_gmsl_tmp_missingdata.append((SSH_at_xy_ano_missingdata*w).sum(skipna=True)/w.sum(skipna=True))
alongtrack_gmsl_missingdata = xr.concat(alongtrack_gmsl_tmp_missingdata, dim='time')

In [28]:
alongtrack_gmsl_missingdata=alongtrack_gmsl_missingdata.drop('cycle_day')
alongtrack_gmsl_missingdata=alongtrack_gmsl_missingdata.drop('delta_days')
alongtrack_gmsl_missingdata.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_missingdata.attrs['units'] = 'm'
alongtrack_gmsl_missingdata.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with missing data, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_missingdata.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_missingdata['time']=alongtrack_gmsl_missingdata.time+(4*24*60*60*10**(9))
alongtrack_gmsl_missingdata

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

In [29]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_missingdata_nomean.nc')
alongtrack_gmsl_missingdata.to_netcdf(fname)

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in arcsin
  return func(*(_execute_task(a, cache) for a in args))
/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/array/core.py:4919: RuntimeWarning: invalid value encountered in float_scalars
  result = function(*args, **kwargs)


### Orbit error - 1cm
#### Compute the reference global mean SSH over 20 years 1996-2016

In [30]:
# ECCO_alongtrack_files=[]
# for year in range(1996,2017):
#     ECCO_alongtrack_files_tmp = list(alongtrack_file_dir_orbiterror.glob('*ECCO_V4r4_alongtrack_SSH_*'+str(year)+'*nc'))
#     ECCO_alongtrack_files.extend(ECCO_alongtrack_files_tmp)
# print(np.sort(ECCO_alongtrack_files))

In [31]:
# alongtrack = xr.open_mfdataset(
#         paths=ECCO_alongtrack_files,
#         combine='nested',
#         concat_dim='i',
#         decode_cf=True,
#         compat='override',
#         coords='minimal',
#         chunks={'time': 1}  
#     )  
# alongtrack

In [32]:
# ref_mean_orbiterror=alongtrack.SSH_at_xy.mean(skipna=True).values
# print(ref_mean_orbiterror)

#### Compute GMSL 10-day time series

In [33]:
ECCO_alongtrack_files = list(alongtrack_file_dir_orbiterror_1cm.glob('*ECCO_V4r4_alongtrack_SSH_*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
print(ECCO_alongtrack_files)

[PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_1cm/ECCO_V4r4_alongtrack_SSH_1992-09-23_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_1cm/ECCO_V4r4_alongtrack_SSH_1992-09-24_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_1cm/ECCO_V4r4_alongtrack_SSH_1992-09-25_orbiterror.nc')
 ...
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_1cm/ECCO_V4r4_alongtrack_SSH_2017-12-29_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_1cm/ECCO_V4r4_alongtrack_SSH_2017-12-30_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_1cm/ECCO_V4r4_alongtrack_SSH_2017-12-31_orbiterror.nc')]


In [34]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    beta = np.pi/2 - gamma
    w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
alongtrack_gmsl_orbiterror_1cm = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [35]:
alongtrack_gmsl_orbiterror_1cm=alongtrack_gmsl_orbiterror_1cm.drop('cycle_day')
alongtrack_gmsl_orbiterror_1cm=alongtrack_gmsl_orbiterror_1cm.drop('delta_days')
alongtrack_gmsl_orbiterror_1cm.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_orbiterror_1cm.attrs['units'] = 'm'
alongtrack_gmsl_orbiterror_1cm.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with 1cm orbit error, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_orbiterror_1cm.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_orbiterror_1cm['time']=alongtrack_gmsl_orbiterror_1cm.time+(4*24*60*60*10**(9))
alongtrack_gmsl_orbiterror_1cm

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

In [36]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_orbiterror_1cm_nomean.nc')
alongtrack_gmsl_orbiterror_1cm.to_netcdf(fname)

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in arcsin
  return func(*(_execute_task(a, cache) for a in args))


### Orbit error - 2cm
#### Compute the reference global mean SSH over 20 years 1996-2016

#### Compute GMSL 10-day time series

In [37]:
ECCO_alongtrack_files = list(alongtrack_file_dir_orbiterror_2cm.glob('*ECCO_V4r4_alongtrack_SSH_*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
print(ECCO_alongtrack_files)

[PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_2cm/ECCO_V4r4_alongtrack_SSH_1992-09-23_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_2cm/ECCO_V4r4_alongtrack_SSH_1992-09-24_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_2cm/ECCO_V4r4_alongtrack_SSH_1992-09-25_orbiterror.nc')
 ...
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_2cm/ECCO_V4r4_alongtrack_SSH_2017-12-29_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_2cm/ECCO_V4r4_alongtrack_SSH_2017-12-30_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_2cm/ECCO_V4r4_alongtrack_SSH_2017-12-31_orbiterror.nc')]


In [38]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    beta = np.pi/2 - gamma
    w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
alongtrack_gmsl_orbiterror_2cm = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [39]:
alongtrack_gmsl_orbiterror_2cm=alongtrack_gmsl_orbiterror_2cm.drop('cycle_day')
alongtrack_gmsl_orbiterror_2cm=alongtrack_gmsl_orbiterror_2cm.drop('delta_days')
alongtrack_gmsl_orbiterror_2cm.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_orbiterror_2cm.attrs['units'] = 'm'
alongtrack_gmsl_orbiterror_2cm.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with 2cm orbit error, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_orbiterror_2cm.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_orbiterror_2cm['time']=alongtrack_gmsl_orbiterror_2cm.time+(4*24*60*60*10**(9))
alongtrack_gmsl_orbiterror_2cm

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

In [40]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_orbiterror_2cm_nomean.nc')
alongtrack_gmsl_orbiterror_2cm.to_netcdf(fname)

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in arcsin
  return func(*(_execute_task(a, cache) for a in args))


### Orbit error - 3cm
#### Compute the reference global mean SSH over 20 years 1996-2016

#### Compute GMSL 10-day time series

In [41]:
ECCO_alongtrack_files = list(alongtrack_file_dir_orbiterror_3cm.glob('*ECCO_V4r4_alongtrack_SSH_*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
print(ECCO_alongtrack_files)

[PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_3cm/ECCO_V4r4_alongtrack_SSH_1992-09-23_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_3cm/ECCO_V4r4_alongtrack_SSH_1992-09-24_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_3cm/ECCO_V4r4_alongtrack_SSH_1992-09-25_orbiterror.nc')
 ...
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_3cm/ECCO_V4r4_alongtrack_SSH_2017-12-29_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_3cm/ECCO_V4r4_alongtrack_SSH_2017-12-30_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_3cm/ECCO_V4r4_alongtrack_SSH_2017-12-31_orbiterror.nc')]


In [42]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    beta = np.pi/2 - gamma
    w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
alongtrack_gmsl_orbiterror_3cm = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [43]:
alongtrack_gmsl_orbiterror_3cm=alongtrack_gmsl_orbiterror_3cm.drop('cycle_day')
alongtrack_gmsl_orbiterror_3cm=alongtrack_gmsl_orbiterror_3cm.drop('delta_days')
alongtrack_gmsl_orbiterror_3cm.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_orbiterror_3cm.attrs['units'] = 'm'
alongtrack_gmsl_orbiterror_3cm.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with 3cm orbit error, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_orbiterror_3cm.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_orbiterror_3cm['time']=alongtrack_gmsl_orbiterror_3cm.time+(4*24*60*60*10**(9))
alongtrack_gmsl_orbiterror_3cm

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

In [44]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_orbiterror_3cm_nomean.nc')
alongtrack_gmsl_orbiterror_3cm.to_netcdf(fname)

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in arcsin
  return func(*(_execute_task(a, cache) for a in args))


### 3 errors: Random noise + missing data + orbit error - 1cm
#### Compute the reference global mean SSH over 20 years 1996-2016

In [45]:
# ECCO_alongtrack_files=[]
# for year in range(1996,2017):
#     ECCO_alongtrack_files_tmp = list(alongtrack_file_dir_orbiterror.glob('*ECCO_V4r4_alongtrack_SSH_*'+str(year)+'*nc'))
#     ECCO_alongtrack_files.extend(ECCO_alongtrack_files_tmp)
# print(np.sort(ECCO_alongtrack_files))

In [46]:
# alongtrack = xr.open_mfdataset(
#         paths=ECCO_alongtrack_files,
#         combine='nested',
#         concat_dim='i',
#         decode_cf=True,
#         compat='override',
#         coords='minimal',
#         chunks={'time': 1}  
#     )  
# alongtrack

In [47]:
# ref_mean_3errors=alongtrack.SSH_at_xy.mean(skipna=True).values
# print(ref_mean_3errors)

#### Compute GMSL 10-day time series

In [48]:
ECCO_alongtrack_files = list(alongtrack_file_dir_3errors_1cm.glob('*ECCO_V4r4_alongtrack_SSH_*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
print(ECCO_alongtrack_files)

[PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_1cm/ECCO_V4r4_alongtrack_SSH_1992-09-23_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_1cm/ECCO_V4r4_alongtrack_SSH_1992-09-24_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_1cm/ECCO_V4r4_alongtrack_SSH_1992-09-25_3errors.nc')
 ...
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_1cm/ECCO_V4r4_alongtrack_SSH_2017-12-29_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_1cm/ECCO_V4r4_alongtrack_SSH_2017-12-30_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_1cm/ECCO_V4r4_alongtrack_SSH_2017-12-31_3errors.nc')]


In [49]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    beta = np.pi/2 - gamma
    w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
alongtrack_gmsl_3errors_1cm = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [50]:
alongtrack_gmsl_3errors_1cm=alongtrack_gmsl_3errors_1cm.drop('cycle_day')
alongtrack_gmsl_3errors_1cm=alongtrack_gmsl_3errors_1cm.drop('delta_days')
alongtrack_gmsl_3errors_1cm.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_3errors_1cm.attrs['units'] = 'm'
alongtrack_gmsl_3errors_1cm.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with 3 errors, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_3errors_1cm.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_3errors_1cm['time']=alongtrack_gmsl_3errors_1cm.time+(4*24*60*60*10**(9))
alongtrack_gmsl_3errors_1cm

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

In [51]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_3errors_1cm_nomean.nc')
alongtrack_gmsl_3errors_1cm.to_netcdf(fname)

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in arcsin
  return func(*(_execute_task(a, cache) for a in args))
/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/array/core.py:4919: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)


### 3 errors: Random noise + missing data + orbit error - 2cm
#### Compute the reference global mean SSH over 20 years 1996-2016

#### Compute GMSL 10-day time series

In [52]:
ECCO_alongtrack_files = list(alongtrack_file_dir_3errors_2cm.glob('*ECCO_V4r4_alongtrack_SSH_*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
print(ECCO_alongtrack_files)

[PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_2cm/ECCO_V4r4_alongtrack_SSH_1992-09-23_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_2cm/ECCO_V4r4_alongtrack_SSH_1992-09-24_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_2cm/ECCO_V4r4_alongtrack_SSH_1992-09-25_3errors.nc')
 ...
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_2cm/ECCO_V4r4_alongtrack_SSH_2017-12-29_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_2cm/ECCO_V4r4_alongtrack_SSH_2017-12-30_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_2cm/ECCO_V4r4_alongtrack_SSH_2017-12-31_3errors.nc')]


In [53]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    beta = np.pi/2 - gamma
    w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
alongtrack_gmsl_3errors_2cm = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [54]:
alongtrack_gmsl_3errors_2cm=alongtrack_gmsl_3errors_2cm.drop('cycle_day')
alongtrack_gmsl_3errors_2cm=alongtrack_gmsl_3errors_2cm.drop('delta_days')
alongtrack_gmsl_3errors_2cm.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_3errors_2cm.attrs['units'] = 'm'
alongtrack_gmsl_3errors_2cm.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with 3 errors, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_3errors_2cm.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_3errors_2cm['time']=alongtrack_gmsl_3errors_2cm.time+(4*24*60*60*10**(9))
alongtrack_gmsl_3errors_2cm

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

In [55]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_3errors_2cm_nomean.nc')
alongtrack_gmsl_3errors_2cm.to_netcdf(fname)

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in arcsin
  return func(*(_execute_task(a, cache) for a in args))
/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/array/core.py:4919: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)


### 3 errors: Random noise + missing data + orbit error - 3cm
#### Compute the reference global mean SSH over 20 years 1996-2016

#### Compute GMSL 10-day time series

In [56]:
ECCO_alongtrack_files = list(alongtrack_file_dir_3errors_3cm.glob('*ECCO_V4r4_alongtrack_SSH_*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
print(ECCO_alongtrack_files)

[PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_3cm/ECCO_V4r4_alongtrack_SSH_1992-09-23_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_3cm/ECCO_V4r4_alongtrack_SSH_1992-09-24_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_3cm/ECCO_V4r4_alongtrack_SSH_1992-09-25_3errors.nc')
 ...
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_3cm/ECCO_V4r4_alongtrack_SSH_2017-12-29_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_3cm/ECCO_V4r4_alongtrack_SSH_2017-12-30_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_3cm/ECCO_V4r4_alongtrack_SSH_2017-12-31_3errors.nc')]


In [57]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    beta = np.pi/2 - gamma
    w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
alongtrack_gmsl_3errors_3cm = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [58]:
alongtrack_gmsl_3errors_3cm=alongtrack_gmsl_3errors_3cm.drop('cycle_day')
alongtrack_gmsl_3errors_3cm=alongtrack_gmsl_3errors_3cm.drop('delta_days')
alongtrack_gmsl_3errors_3cm.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_3errors_3cm.attrs['units'] = 'm'
alongtrack_gmsl_3errors_3cm.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with 3 errors, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_3errors_3cm.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_3errors_3cm['time']=alongtrack_gmsl_3errors_3cm.time+(4*24*60*60*10**(9))
alongtrack_gmsl_3errors_3cm

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

In [59]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_3errors_3cm_nomean.nc')
alongtrack_gmsl_3errors_3cm.to_netcdf(fname)

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in arcsin
  return func(*(_execute_task(a, cache) for a in args))
/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/array/core.py:4919: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)


# Compute GMSL from ECCO synthetic grids with simple weighing

### No error

In [60]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_2017-12-25.nc')]


In [61]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 360, time: 924)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [62]:
grids=grids.where(np.array(grids.counts)>=100)

In [63]:
grids=grids.sel(latitude=slice(-66,66))
grids

<xarray.Dataset>
Dimensions:    (time: 924, latitude: 264, longitude: 720)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [64]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [65]:
# make new DataArray objects
gmsl_grids = xr.DataArray(ssh_weighted_mean, dims=['time'])
gmsl_grids = gmsl_grids.assign_coords({'time':grids.time})
gmsl_grids.name = 'gmsl_grids'
gmsl_grids

<xarray.DataArray 'gmsl_grids' (time: 924)>
array([-3.42057087e-02, -3.17938551e-02, -3.01074404e-02, -3.06746941e-02,
       -3.16903107e-02, -3.39044407e-02, -3.15182321e-02, -3.33229601e-02,
       -3.32409255e-02, -3.91682088e-02, -4.09580506e-02, -4.04708274e-02,
       -4.08578888e-02, -4.26068678e-02, -4.31960486e-02, -3.99494469e-02,
       -3.98812965e-02, -3.81741263e-02, -4.10182253e-02, -3.91411074e-02,
       -3.83483097e-02, -3.76450084e-02, -3.88881490e-02, -3.90859805e-02,
       -3.76473330e-02, -3.70767415e-02, -3.80722657e-02, -3.75312530e-02,
       -3.93981710e-02, -3.78221571e-02, -3.66856903e-02, -3.73637453e-02,
       -3.52189876e-02, -3.39760669e-02, -3.19755413e-02, -3.22776362e-02,
       -3.04331351e-02, -2.85196956e-02, -2.75158118e-02, -2.76557654e-02,
       -2.97151413e-02, -3.07964608e-02, -3.35024446e-02, -3.24334912e-02,
       -2.88796332e-02, -2.69621406e-02, -3.14496420e-02, -3.27305868e-02,
       -3.04761846e-02, -3.41623388e-02, -3.85107137e-02, -4.00159694e-02,
       -3.54074687e-02, -3.30436118e-02, -3.23276147e-02, -3.33706215e-02,
       -3.17507237e-02, -3.38620394e-02, -3.34102176e-02, -3.68275121e-02,
       -3.63152847e-02, -3.57615761e-02, -3.88607830e-02, -4.00873274e-02,
       -4.11999859e-02, -3.96811552e-02, -3.77491489e-02, -3.70679870e-02,
       -3.56467180e-02, -3.60416956e-02, -3.24518122e-02, -3.19891423e-02,
       -2.99776476e-02, -3.06052081e-02, -2.71528885e-02, -2.78811324e-02,
       -2.62111407e-02, -2.68377755e-02, -2.87042502e-02, -2.91528441e-02,
...
        4.33473177e-02,  3.92972082e-02,  3.75151932e-02,  4.10812981e-02,
        4.13174219e-02,  3.74017172e-02,  3.80713716e-02,  3.64492573e-02,
        3.80542688e-02,  3.55226286e-02,  3.85362655e-02,  3.63907702e-02,
        3.73638719e-02,  3.67255695e-02,  3.89472730e-02,  3.82025614e-02,
        3.68385427e-02,  3.80890071e-02,  3.70339565e-02,  3.68256122e-02,
        3.74539942e-02,  3.63551639e-02,  3.57640460e-02,  3.94382514e-02,
        3.94841023e-02,  4.04666439e-02,  4.28168736e-02,  4.46808562e-02,
        4.24690023e-02,  4.19718325e-02,  4.39331532e-02,  4.67256755e-02,
        4.62449417e-02,  4.77000065e-02,  4.69091013e-02,  4.83014509e-02,
        4.64458056e-02,  4.59901653e-02,  4.44731303e-02,  4.45664227e-02,
        4.21740301e-02,  3.84246595e-02,  3.63126062e-02,  3.70804556e-02,
        3.69437523e-02,  3.75306569e-02,  3.75632010e-02,  3.73779908e-02,
        3.88500951e-02,  3.98654826e-02,  4.08814400e-02,  4.12371121e-02,
        3.98347452e-02,  3.92047018e-02,  4.20654975e-02,  4.13155183e-02,
        4.15603630e-02,  4.00834531e-02,  4.06483561e-02,  4.41597812e-02,
        4.44199704e-02,  4.69005927e-02,  4.74085137e-02,  4.75556590e-02,
        5.01045659e-02,  4.74308617e-02,  4.80947271e-02,  4.86356840e-02,
        5.14050946e-02,  5.33416979e-02,  5.38304783e-02,  5.66883832e-02,
        5.44943325e-02,  5.53821288e-02,  5.28612845e-02,  5.45522012e-02],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### Random noise

In [66]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_randomnoise.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_2017-12-25.nc')]


In [67]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 360, time: 924)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [68]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66))
grids

<xarray.Dataset>
Dimensions:    (time: 924, latitude: 264, longitude: 720)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [69]:
# weights = np.cos(np.deg2rad(grids.latitude))
# weights.name = "weights"
# weights

In [70]:
# ssh_weighted = grids.weighted(weights,skipna=True)
# ssh_weighted

In [71]:
# ssh_weighted_mean_randomnoise = ssh_weighted.mean(("longitude", "latitude"),skipna=True)
# ssh_weighted_mean_randomnoise.SSHA

In [72]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_randomnoise=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [73]:
# make new DataArray objects
gmsl_grids_randomnoise = xr.DataArray(ssh_weighted_mean_randomnoise, dims=['time'])
gmsl_grids_randomnoise = gmsl_grids_randomnoise.assign_coords({'time':grids.time})
gmsl_grids_randomnoise.name = 'gmsl_grids'
gmsl_grids_randomnoise

<xarray.DataArray 'gmsl_grids' (time: 924)>
array([-3.41900587e-02, -3.18217091e-02, -3.00949700e-02, -3.06222495e-02,
       -3.16870883e-02, -3.38858403e-02, -3.15365605e-02, -3.33235301e-02,
       -3.32720242e-02, -3.90845314e-02, -4.09118123e-02, -4.04039435e-02,
       -4.08666171e-02, -4.25825864e-02, -4.31476422e-02, -3.98971774e-02,
       -3.98596823e-02, -3.81896161e-02, -4.09829468e-02, -3.91657278e-02,
       -3.82959321e-02, -3.76530252e-02, -3.88898104e-02, -3.89906988e-02,
       -3.76198702e-02, -3.70374471e-02, -3.80518623e-02, -3.74453217e-02,
       -3.94491851e-02, -3.77553664e-02, -3.66845131e-02, -3.73321362e-02,
       -3.52189802e-02, -3.39215323e-02, -3.20342146e-02, -3.23095210e-02,
       -3.03761680e-02, -2.85412762e-02, -2.74800826e-02, -2.76653655e-02,
       -2.97073387e-02, -3.07414234e-02, -3.35180312e-02, -3.24415527e-02,
       -2.89238952e-02, -2.69511454e-02, -3.14745344e-02, -3.26978639e-02,
       -3.05241924e-02, -3.41811702e-02, -3.85051630e-02, -4.00350355e-02,
       -3.54452953e-02, -3.30273397e-02, -3.23136263e-02, -3.34170386e-02,
       -3.17076929e-02, -3.39140072e-02, -3.33579071e-02, -3.67526151e-02,
       -3.63572873e-02, -3.57552990e-02, -3.88489515e-02, -4.01663929e-02,
       -4.11581360e-02, -3.97008918e-02, -3.77787128e-02, -3.71138528e-02,
       -3.56613621e-02, -3.59894708e-02, -3.24606858e-02, -3.20110694e-02,
       -2.99666151e-02, -3.06757260e-02, -2.72889398e-02, -2.78788675e-02,
       -2.61677131e-02, -2.68284548e-02, -2.86806710e-02, -2.91644186e-02,
...
        4.33735400e-02,  3.93408276e-02,  3.74733508e-02,  4.11022492e-02,
        4.12392579e-02,  3.74163091e-02,  3.80750187e-02,  3.65108028e-02,
        3.80382128e-02,  3.54800895e-02,  3.85831445e-02,  3.64350714e-02,
        3.74226868e-02,  3.67228799e-02,  3.89234461e-02,  3.82459350e-02,
        3.68058793e-02,  3.80691700e-02,  3.70368622e-02,  3.68240103e-02,
        3.74514647e-02,  3.63438614e-02,  3.57098840e-02,  3.95411439e-02,
        3.95294316e-02,  4.04621661e-02,  4.27543782e-02,  4.46532965e-02,
        4.25370522e-02,  4.19330932e-02,  4.38353233e-02,  4.66419458e-02,
        4.63016853e-02,  4.77225408e-02,  4.68652844e-02,  4.82493117e-02,
        4.64512259e-02,  4.61082235e-02,  4.44222353e-02,  4.45657521e-02,
        4.21717465e-02,  3.84297669e-02,  3.63522768e-02,  3.70551273e-02,
        3.69827375e-02,  3.75184529e-02,  3.76403071e-02,  3.74321863e-02,
        3.88421342e-02,  3.98085937e-02,  4.08548005e-02,  4.12295535e-02,
        3.98259498e-02,  3.92172262e-02,  4.20877077e-02,  4.13332656e-02,
        4.15127426e-02,  4.01592068e-02,  4.06106785e-02,  4.41554002e-02,
        4.43885028e-02,  4.68618087e-02,  4.74343076e-02,  4.75702621e-02,
        5.01110069e-02,  4.74485271e-02,  4.82227206e-02,  4.86221984e-02,
        5.14656715e-02,  5.33827767e-02,  5.38228825e-02,  5.66830374e-02,
        5.44360392e-02,  5.54379821e-02,  5.28247356e-02,  5.43186739e-02],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### Missing data

In [74]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_missingdata.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_2017-12-25.nc')]


In [75]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 360, time: 923)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [76]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66))
grids

<xarray.Dataset>
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [77]:
# weights = np.cos(np.deg2rad(grids.latitude))
# weights.name = "weights"
# weights

In [78]:
# ssh_weighted = grids.weighted(weights,skipna=True)
# ssh_weighted

In [79]:
# ssh_weighted_mean_missingdata = ssh_weighted.mean(("longitude", "latitude"),skipna=True)
# ssh_weighted_mean_missingdata.SSHA

In [80]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_missingdata=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [81]:
# make new DataArray objects
gmsl_grids_missingdata = xr.DataArray(ssh_weighted_mean_missingdata, dims=['time'])
gmsl_grids_missingdata = gmsl_grids_missingdata.assign_coords({'time':grids.time})
gmsl_grids_missingdata.name = 'gmsl_grids'
gmsl_grids_missingdata

<xarray.DataArray 'gmsl_grids' (time: 923)>
array([-3.39181721e-02, -3.17041986e-02, -3.01630292e-02, -3.07401959e-02,
       -3.17151695e-02, -3.40163931e-02, -3.15430872e-02, -3.35007422e-02,
       -3.33031043e-02, -3.91796976e-02, -4.09364849e-02, -4.04163748e-02,
       -4.09599207e-02, -4.25666161e-02, -4.31523174e-02, -4.01557274e-02,
       -3.99127379e-02, -3.81559394e-02, -4.10287715e-02, -3.90594937e-02,
       -3.83507125e-02, -3.76562588e-02, -3.88363600e-02, -3.90506387e-02,
       -3.76322754e-02, -3.70418020e-02, -3.80836576e-02, -3.75130065e-02,
       -3.92687768e-02, -3.77710611e-02, -3.53762731e-02, -3.72993909e-02,
       -3.51879597e-02, -3.37432735e-02, -3.19958776e-02, -3.22780050e-02,
       -3.04171238e-02, -2.84203421e-02, -2.74890102e-02, -2.76016630e-02,
       -2.97415629e-02, -3.08316071e-02, -3.35056297e-02, -3.20282467e-02,
       -2.89579295e-02, -2.69683432e-02, -3.14311869e-02, -3.27173956e-02,
       -3.03644910e-02, -3.41236070e-02, -3.85808162e-02, -4.00847085e-02,
       -3.54501754e-02, -3.30323614e-02, -3.24828401e-02, -3.34091485e-02,
       -3.17647457e-02, -3.38546485e-02, -3.33819650e-02, -3.68301980e-02,
       -3.62913683e-02, -3.56815308e-02, -3.88499647e-02, -4.00284119e-02,
       -4.12970856e-02, -3.95743251e-02, -3.77149433e-02, -3.70792970e-02,
       -3.56211253e-02, -3.60497460e-02, -3.24732587e-02, -3.19635421e-02,
       -2.98006926e-02, -3.05586569e-02, -2.70284321e-02, -2.79066265e-02,
       -2.61723604e-02, -2.68666949e-02, -2.90700141e-02, -2.91701108e-02,
...
        4.59035598e-02,  4.74180095e-02,  4.44946401e-02,  4.30879593e-02,
        3.92126553e-02,  3.74304838e-02,  4.11800072e-02,  4.11464311e-02,
        3.73855308e-02,  3.81070152e-02,  3.63713764e-02,  3.80423255e-02,
        3.55643369e-02,  3.85409929e-02,  3.63063850e-02,  3.73827443e-02,
        3.67483348e-02,  3.89066637e-02,  3.82128060e-02,  3.69137302e-02,
        3.80512066e-02,  3.71226408e-02,  3.69269848e-02,  3.75294127e-02,
        3.65217328e-02,  3.58469971e-02,  3.95150296e-02,  3.95167693e-02,
        4.04826328e-02,  4.28142957e-02,  4.47865501e-02,  4.24439311e-02,
        4.19457629e-02,  4.39372100e-02,  4.66874167e-02,  4.62952517e-02,
        4.77353223e-02,  4.68878560e-02,  4.84487265e-02,  4.63270247e-02,
        4.59648967e-02,  4.45173308e-02,  4.45506498e-02,  4.18926775e-02,
        3.84333208e-02,  3.63229662e-02,  3.73353995e-02,  3.69916670e-02,
        3.74541581e-02,  3.75537984e-02,  3.73927839e-02,  3.88507433e-02,
        3.98637652e-02,  4.08880822e-02,  4.11935262e-02,  3.98042686e-02,
        3.92188430e-02,  4.20104042e-02,  4.12083119e-02,  4.13358249e-02,
        4.00069728e-02,  4.07791957e-02,  4.40981574e-02,  4.43659537e-02,
        4.68985103e-02,  4.72977608e-02,  4.75111566e-02,  5.01150265e-02,
        4.74370793e-02,  4.80728261e-02,  4.85891551e-02,  5.14542349e-02,
        5.33894785e-02,  5.37371524e-02,  5.66199794e-02,  5.45640662e-02,
        5.54397106e-02,  5.26633970e-02,  5.40077239e-02], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### Orbit Error - 1cm

In [82]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_orbiterror_1cm.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_1cm/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_1cm/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_1cm/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_1cm/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_1cm/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_1cm/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_1cm/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_1cm/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_1cm/SSHA_gridded_2017-12-25.nc')]


In [83]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 360, time: 924)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [84]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66))
grids

<xarray.Dataset>
Dimensions:    (time: 924, latitude: 264, longitude: 720)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [85]:
# weights = np.cos(np.deg2rad(grids.latitude))
# weights.name = "weights"
# weights

In [86]:
# ssh_weighted = grids.weighted(weights,skipna=True)
# ssh_weighted

In [87]:
# ssh_weighted_mean_orbiterror = ssh_weighted.mean(("longitude", "latitude"),skipna=True)
# ssh_weighted_mean_orbiterror.SSHA

In [88]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_orbiterror=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [89]:
# make new DataArray objects
gmsl_grids_orbiterror_1cm = xr.DataArray(ssh_weighted_mean_orbiterror, dims=['time'])
gmsl_grids_orbiterror_1cm = gmsl_grids_orbiterror_1cm.assign_coords({'time':grids.time})
gmsl_grids_orbiterror_1cm.name = 'gmsl_grids'
gmsl_grids_orbiterror_1cm

<xarray.DataArray 'gmsl_grids' (time: 924)>
array([-3.42807733e-02, -3.18251476e-02, -3.00103649e-02, -3.10092196e-02,
       -3.17050666e-02, -3.39634232e-02, -3.17015424e-02, -3.35945897e-02,
       -3.33265141e-02, -3.94432358e-02, -4.10359688e-02, -4.03606109e-02,
       -4.09268402e-02, -4.23263796e-02, -4.32395525e-02, -3.97875346e-02,
       -3.97993848e-02, -3.79942544e-02, -4.09765989e-02, -3.94268855e-02,
       -3.84605788e-02, -3.76644805e-02, -3.86729911e-02, -3.91882248e-02,
       -3.75032686e-02, -3.69069241e-02, -3.82049531e-02, -3.73475812e-02,
       -3.94573398e-02, -3.74333486e-02, -3.68925706e-02, -3.74060757e-02,
       -3.55201475e-02, -3.40671726e-02, -3.18928733e-02, -3.25427577e-02,
       -3.04811113e-02, -2.83083580e-02, -2.73840781e-02, -2.78664920e-02,
       -2.95699276e-02, -3.09770498e-02, -3.35551463e-02, -3.25515382e-02,
       -2.85600312e-02, -2.68252417e-02, -3.12094670e-02, -3.26019339e-02,
       -3.04961801e-02, -3.45694125e-02, -3.84446084e-02, -4.01434042e-02,
       -3.54026407e-02, -3.33817899e-02, -3.21606807e-02, -3.36446464e-02,
       -3.16584706e-02, -3.42166536e-02, -3.36718671e-02, -3.66788395e-02,
       -3.60255204e-02, -3.58521193e-02, -3.89340222e-02, -3.98801714e-02,
       -4.12420668e-02, -3.96631062e-02, -3.80231403e-02, -3.70454825e-02,
       -3.59448679e-02, -3.60249951e-02, -3.24430875e-02, -3.20675969e-02,
       -2.99990643e-02, -3.04924734e-02, -2.72893179e-02, -2.77536847e-02,
       -2.62368452e-02, -2.70855315e-02, -2.87639014e-02, -2.91392952e-02,
...
        4.33386192e-02,  3.93888913e-02,  3.72830592e-02,  4.07942720e-02,
        4.13179323e-02,  3.74478735e-02,  3.81267890e-02,  3.64117622e-02,
        3.80570851e-02,  3.53048891e-02,  3.87987457e-02,  3.64114866e-02,
        3.72950397e-02,  3.65787446e-02,  3.90959345e-02,  3.81268561e-02,
        3.68956216e-02,  3.82362157e-02,  3.68465520e-02,  3.63567024e-02,
        3.73574272e-02,  3.63970734e-02,  3.58039439e-02,  3.96525487e-02,
        3.91166657e-02,  4.04714756e-02,  4.28839177e-02,  4.46703285e-02,
        4.21332754e-02,  4.21031453e-02,  4.39424962e-02,  4.64475006e-02,
        4.63532917e-02,  4.79274914e-02,  4.68663387e-02,  4.82549220e-02,
        4.62303273e-02,  4.58693244e-02,  4.45083380e-02,  4.45782505e-02,
        4.20705043e-02,  3.83556262e-02,  3.60854603e-02,  3.69575992e-02,
        3.70892398e-02,  3.76088247e-02,  3.72646227e-02,  3.75650525e-02,
        3.88006940e-02,  4.00053225e-02,  4.08272482e-02,  4.11158651e-02,
        3.98254171e-02,  3.90303284e-02,  4.18362729e-02,  4.12243009e-02,
        4.17048894e-02,  4.00279798e-02,  4.05674092e-02,  4.38707918e-02,
        4.43932898e-02,  4.69255671e-02,  4.71943542e-02,  4.76979762e-02,
        5.06752916e-02,  4.71900105e-02,  4.82294783e-02,  4.85647842e-02,
        5.11208400e-02,  5.32376878e-02,  5.38686365e-02,  5.66977598e-02,
        5.43943457e-02,  5.54790683e-02,  5.27638942e-02,  5.42635955e-02],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### Orbit Error - 2cm

In [90]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_orbiterror_2cm.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_2cm/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_2cm/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_2cm/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_2cm/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_2cm/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_2cm/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_2cm/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_2cm/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_2cm/SSHA_gridded_2017-12-25.nc')]


In [91]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 360, time: 924)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [92]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66))
grids

<xarray.Dataset>
Dimensions:    (time: 924, latitude: 264, longitude: 720)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [93]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_orbiterror=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [94]:
# make new DataArray objects
gmsl_grids_orbiterror_2cm = xr.DataArray(ssh_weighted_mean_orbiterror, dims=['time'])
gmsl_grids_orbiterror_2cm = gmsl_grids_orbiterror_2cm.assign_coords({'time':grids.time})
gmsl_grids_orbiterror_2cm.name = 'gmsl_grids'
gmsl_grids_orbiterror_2cm

<xarray.DataArray 'gmsl_grids' (time: 924)>
array([-3.47389728e-02, -3.14447023e-02, -3.00595853e-02, -3.04746628e-02,
       -3.11883017e-02, -3.38848233e-02, -3.11407670e-02, -3.36561091e-02,
       -3.37947346e-02, -3.91617939e-02, -4.15213369e-02, -4.08004262e-02,
       -4.15183604e-02, -4.27081101e-02, -4.32537720e-02, -3.96177024e-02,
       -3.98899503e-02, -3.83457057e-02, -4.12799753e-02, -3.91572081e-02,
       -3.88413072e-02, -3.73919383e-02, -3.85700315e-02, -3.90499718e-02,
       -3.79125997e-02, -3.66899297e-02, -3.92029397e-02, -3.73575501e-02,
       -3.90644968e-02, -3.73024940e-02, -3.67243923e-02, -3.71007845e-02,
       -3.49668749e-02, -3.37925665e-02, -3.22848521e-02, -3.22024748e-02,
       -3.04304138e-02, -2.80916672e-02, -2.73724422e-02, -2.77675837e-02,
       -2.92122923e-02, -3.05859018e-02, -3.36623229e-02, -3.20721194e-02,
       -2.93424688e-02, -2.69902479e-02, -3.14601138e-02, -3.26679759e-02,
       -3.00100744e-02, -3.47510874e-02, -3.91110368e-02, -4.07261960e-02,
       -3.52839343e-02, -3.33317630e-02, -3.21023352e-02, -3.32178473e-02,
       -3.19506377e-02, -3.47220600e-02, -3.35787907e-02, -3.63245606e-02,
       -3.63172144e-02, -3.61537971e-02, -3.91448326e-02, -3.96701656e-02,
       -4.15218771e-02, -3.95453535e-02, -3.72103937e-02, -3.76357511e-02,
       -3.53554487e-02, -3.63624617e-02, -3.25017832e-02, -3.21748927e-02,
       -2.98384950e-02, -3.09401974e-02, -2.67119929e-02, -2.68914308e-02,
       -2.58747861e-02, -2.63994094e-02, -2.83898134e-02, -2.91509070e-02,
...
        4.30709347e-02,  3.95717956e-02,  3.74398604e-02,  4.08220142e-02,
        4.18165848e-02,  3.71300280e-02,  3.82387787e-02,  3.61693986e-02,
        3.74294929e-02,  3.51074636e-02,  3.88054475e-02,  3.64742838e-02,
        3.67336385e-02,  3.69375832e-02,  3.87937054e-02,  3.82139981e-02,
        3.70241106e-02,  3.83206792e-02,  3.70555669e-02,  3.68690044e-02,
        3.72783765e-02,  3.65152992e-02,  3.60498428e-02,  3.92251611e-02,
        3.97023857e-02,  4.06759456e-02,  4.28643450e-02,  4.49610688e-02,
        4.27726433e-02,  4.16450687e-02,  4.36561666e-02,  4.57052849e-02,
        4.62615862e-02,  4.81177159e-02,  4.70288210e-02,  4.90061641e-02,
        4.65453193e-02,  4.57004309e-02,  4.46910635e-02,  4.41383682e-02,
        4.18677218e-02,  3.86055186e-02,  3.66885029e-02,  3.69786844e-02,
        3.73828001e-02,  3.77487615e-02,  3.77018601e-02,  3.78046706e-02,
        3.91891859e-02,  3.96892056e-02,  4.15956080e-02,  4.06379178e-02,
        3.95141803e-02,  3.96897718e-02,  4.21886481e-02,  4.12811972e-02,
        4.09612730e-02,  4.02233563e-02,  4.06528078e-02,  4.41347510e-02,
        4.48745154e-02,  4.72003296e-02,  4.68474962e-02,  4.71889079e-02,
        5.02309315e-02,  4.72193249e-02,  4.74846549e-02,  4.90534194e-02,
        5.15885279e-02,  5.34909330e-02,  5.41565157e-02,  5.67565151e-02,
        5.46029992e-02,  5.58798276e-02,  5.26624471e-02,  5.50156049e-02],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### Orbit Error - 3cm

In [95]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_orbiterror_3cm.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_3cm/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_3cm/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_3cm/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_3cm/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_3cm/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_3cm/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_3cm/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_3cm/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_3cm/SSHA_gridded_2017-12-25.nc')]


In [96]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 360, time: 924)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [97]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66))
grids

<xarray.Dataset>
Dimensions:    (time: 924, latitude: 264, longitude: 720)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [98]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_orbiterror=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [99]:
# make new DataArray objects
gmsl_grids_orbiterror_3cm = xr.DataArray(ssh_weighted_mean_orbiterror, dims=['time'])
gmsl_grids_orbiterror_3cm = gmsl_grids_orbiterror_3cm.assign_coords({'time':grids.time})
gmsl_grids_orbiterror_3cm.name = 'gmsl_grids'
gmsl_grids_orbiterror_3cm

<xarray.DataArray 'gmsl_grids' (time: 924)>
array([-3.44223417e-02, -3.14967893e-02, -3.07295211e-02, -3.15467641e-02,
       -3.16758677e-02, -3.28462198e-02, -3.13428231e-02, -3.30508649e-02,
       -3.32864150e-02, -3.85921299e-02, -4.05393466e-02, -4.01074328e-02,
       -4.13160324e-02, -4.32249457e-02, -4.30941731e-02, -3.98896486e-02,
       -3.94248031e-02, -3.69776636e-02, -4.10320684e-02, -3.90206091e-02,
       -3.83597352e-02, -3.79988551e-02, -3.79506722e-02, -3.91136482e-02,
       -3.62416506e-02, -3.65041867e-02, -3.79863791e-02, -3.77957076e-02,
       -3.91118675e-02, -3.73046324e-02, -3.74189653e-02, -3.77611071e-02,
       -3.38705890e-02, -3.36495303e-02, -3.26283462e-02, -3.17718275e-02,
       -3.06255650e-02, -2.83836946e-02, -2.60830186e-02, -2.72361562e-02,
       -2.97469534e-02, -3.02976798e-02, -3.32269035e-02, -3.21488716e-02,
       -2.92143505e-02, -2.65104007e-02, -3.12616192e-02, -3.29751819e-02,
       -3.09032276e-02, -3.42151113e-02, -3.83649990e-02, -4.04748991e-02,
       -3.55252288e-02, -3.36302258e-02, -3.25700343e-02, -3.37262601e-02,
       -3.19210663e-02, -3.38649452e-02, -3.29965204e-02, -3.66454124e-02,
       -3.62634100e-02, -3.60250957e-02, -3.90542001e-02, -4.08499353e-02,
       -4.06525470e-02, -4.01133299e-02, -3.70011665e-02, -3.71208228e-02,
       -3.60039957e-02, -3.64144780e-02, -3.30700688e-02, -3.21195908e-02,
       -3.02046239e-02, -2.92448774e-02, -2.67881639e-02, -2.76742838e-02,
       -2.58248392e-02, -2.67816354e-02, -2.89800428e-02, -2.90009566e-02,
...
        4.31641378e-02,  3.85281295e-02,  3.75487097e-02,  4.07187641e-02,
        4.07077894e-02,  3.75507213e-02,  3.74244750e-02,  3.63030173e-02,
        3.79272290e-02,  3.56404521e-02,  3.88969444e-02,  3.61948870e-02,
        3.73040922e-02,  3.65673453e-02,  3.89607213e-02,  3.86900976e-02,
        3.63996364e-02,  3.83265167e-02,  3.73777337e-02,  3.72025855e-02,
        3.83161493e-02,  3.64379399e-02,  3.58419530e-02,  3.91496420e-02,
        3.96115333e-02,  3.98276113e-02,  4.30304594e-02,  4.44895253e-02,
        4.19715531e-02,  4.14445698e-02,  4.39042188e-02,  4.86151576e-02,
        4.63590361e-02,  4.74188253e-02,  4.67132330e-02,  4.83108386e-02,
        4.63797525e-02,  4.64598350e-02,  4.45861965e-02,  4.45544794e-02,
        4.22548242e-02,  3.86360213e-02,  3.64624076e-02,  3.74258384e-02,
        3.71491387e-02,  3.75037454e-02,  3.71486247e-02,  3.73492949e-02,
        3.81479487e-02,  3.98623422e-02,  4.07494120e-02,  4.00204249e-02,
        3.89530435e-02,  3.99531312e-02,  4.17696722e-02,  4.10593972e-02,
        4.02724445e-02,  3.97196375e-02,  4.10031602e-02,  4.45816256e-02,
        4.42366265e-02,  4.75260466e-02,  4.57822271e-02,  4.76296246e-02,
        4.96396907e-02,  4.85605411e-02,  4.83902171e-02,  4.81558815e-02,
        5.15759960e-02,  5.26835360e-02,  5.36819920e-02,  5.56316450e-02,
        5.42045310e-02,  5.52692451e-02,  5.19242473e-02,  5.49743101e-02],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### 3 errors: random noise + missing data + orbit error - 1cm

In [100]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_3errors_1cm.glob('*SSHA_gridded*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_1cm/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_1cm/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_1cm/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_1cm/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_1cm/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_1cm/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_1cm/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_1cm/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_1cm/SSHA_gridded_2017-12-25.nc')]


In [101]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 360, time: 923)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [102]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66))
grids

<xarray.Dataset>
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [103]:
# weights = np.cos(np.deg2rad(grids.latitude))
# weights.name = "weights"
# weights

In [104]:
# ssh_weighted = grids.weighted(weights,skipna=True)
# ssh_weighted

In [105]:
# ssh_weighted_mean_3errors = ssh_weighted.mean(("longitude", "latitude"),skipna=True)
# ssh_weighted_mean_3errors.SSHA

In [106]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_3errors=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [107]:
# make new DataArray objects
gmsl_grids_3errors_1cm = xr.DataArray(ssh_weighted_mean_3errors, dims=['time'])
gmsl_grids_3errors_1cm = gmsl_grids_3errors_1cm.assign_coords({'time':grids.time})
gmsl_grids_3errors_1cm.name = 'gmsl_grids'
gmsl_grids_3errors_1cm

<xarray.DataArray 'gmsl_grids' (time: 923)>
array([-3.38551626e-02, -3.18890624e-02, -3.01028267e-02, -3.08667552e-02,
       -3.19170319e-02, -3.42820659e-02, -3.15149017e-02, -3.34871598e-02,
       -3.34158838e-02, -3.89333926e-02, -4.08302508e-02, -4.04792950e-02,
       -4.10161912e-02, -4.25513089e-02, -4.32247669e-02, -3.99807058e-02,
       -3.96748930e-02, -3.82843837e-02, -4.11476754e-02, -3.89713608e-02,
       -3.81547622e-02, -3.79803404e-02, -3.91193293e-02, -3.86192948e-02,
       -3.78147550e-02, -3.67609113e-02, -3.78736071e-02, -3.73528302e-02,
       -3.94268632e-02, -3.78626063e-02, -3.51247862e-02, -3.73236239e-02,
       -3.51538882e-02, -3.38393152e-02, -3.20099816e-02, -3.20808776e-02,
       -3.05750500e-02, -2.83261109e-02, -2.75877602e-02, -2.75378153e-02,
       -2.98306048e-02, -3.12080346e-02, -3.32610868e-02, -3.18812840e-02,
       -2.87631396e-02, -2.69872919e-02, -3.14418301e-02, -3.24790068e-02,
       -3.05656586e-02, -3.39919813e-02, -3.85934971e-02, -4.02211435e-02,
       -3.55848148e-02, -3.34038027e-02, -3.26636322e-02, -3.37254480e-02,
       -3.17608044e-02, -3.38753574e-02, -3.35690528e-02, -3.67021114e-02,
       -3.65188569e-02, -3.58634815e-02, -3.86716872e-02, -3.99899408e-02,
       -4.13376316e-02, -3.94323431e-02, -3.75728719e-02, -3.68148051e-02,
       -3.56035978e-02, -3.61887701e-02, -3.25330086e-02, -3.21074612e-02,
       -2.97839995e-02, -3.06238011e-02, -2.67461315e-02, -2.77445205e-02,
       -2.64235660e-02, -2.68331058e-02, -2.89467163e-02, -2.94507686e-02,
...
        4.59550805e-02,  4.73356918e-02,  4.43296954e-02,  4.30660248e-02,
        3.92656028e-02,  3.74298841e-02,  4.10403237e-02,  4.08644937e-02,
        3.72458734e-02,  3.84230316e-02,  3.61587852e-02,  3.79909314e-02,
        3.56384963e-02,  3.83213758e-02,  3.64685357e-02,  3.75578068e-02,
        3.66168879e-02,  3.90333310e-02,  3.84632386e-02,  3.69820185e-02,
        3.80346067e-02,  3.69096138e-02,  3.66100557e-02,  3.76633704e-02,
        3.62671018e-02,  3.59711125e-02,  3.96329574e-02,  3.94351296e-02,
        4.07137200e-02,  4.26679924e-02,  4.48697321e-02,  4.23487127e-02,
        4.21572365e-02,  4.35422435e-02,  4.66687605e-02,  4.64488193e-02,
        4.75890189e-02,  4.69046421e-02,  4.83671911e-02,  4.62830588e-02,
        4.60758284e-02,  4.47170921e-02,  4.49473746e-02,  4.17037420e-02,
        3.83268148e-02,  3.62776071e-02,  3.73718590e-02,  3.72743569e-02,
        3.73984799e-02,  3.74606811e-02,  3.75401042e-02,  3.88037153e-02,
        3.99272256e-02,  4.12283801e-02,  4.14619744e-02,  3.97827327e-02,
        3.91976945e-02,  4.20841612e-02,  4.13944684e-02,  4.12803367e-02,
        4.02231999e-02,  4.08293977e-02,  4.41089161e-02,  4.46996763e-02,
        4.70756032e-02,  4.69533503e-02,  4.73018996e-02,  5.01510128e-02,
        4.75498363e-02,  4.80772629e-02,  4.86528985e-02,  5.14810607e-02,
        5.35157695e-02,  5.37286624e-02,  5.67164645e-02,  5.44671640e-02,
        5.54137900e-02,  5.28715402e-02,  5.40395230e-02], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### 3 errors: random noise + missing data + orbit error - 2cm

In [108]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_3errors_2cm.glob('*SSHA_gridded*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_2cm/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_2cm/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_2cm/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_2cm/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_2cm/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_2cm/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_2cm/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_2cm/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_2cm/SSHA_gridded_2017-12-25.nc')]


In [109]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 360, time: 923)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [110]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66))
grids

<xarray.Dataset>
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [111]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_3errors=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [112]:
# make new DataArray objects
gmsl_grids_3errors_2cm = xr.DataArray(ssh_weighted_mean_3errors, dims=['time'])
gmsl_grids_3errors_2cm = gmsl_grids_3errors_2cm.assign_coords({'time':grids.time})
gmsl_grids_3errors_2cm.name = 'gmsl_grids'
gmsl_grids_3errors_2cm

<xarray.DataArray 'gmsl_grids' (time: 923)>
array([-3.35063301e-02, -3.20450142e-02, -3.00198160e-02, -3.04576103e-02,
       -3.15192193e-02, -3.41467895e-02, -3.21058035e-02, -3.34657393e-02,
       -3.31944004e-02, -3.91839556e-02, -4.09354568e-02, -4.09188271e-02,
       -4.14052755e-02, -4.28660586e-02, -4.28182706e-02, -4.06709537e-02,
       -4.00703736e-02, -3.81357446e-02, -4.06333953e-02, -3.90820615e-02,
       -3.80957052e-02, -3.80814597e-02, -3.81753966e-02, -4.01522070e-02,
       -3.76516841e-02, -3.63969654e-02, -3.88208665e-02, -3.71955335e-02,
       -3.97218727e-02, -3.74995060e-02, -3.49904373e-02, -3.68597321e-02,
       -3.51504609e-02, -3.39648761e-02, -3.17446925e-02, -3.26102227e-02,
       -3.06181014e-02, -2.83298604e-02, -2.79349834e-02, -2.75497455e-02,
       -3.03388052e-02, -3.10307089e-02, -3.31530459e-02, -3.19483764e-02,
       -2.89414153e-02, -2.70500220e-02, -3.14486250e-02, -3.28307264e-02,
       -3.05474624e-02, -3.41521166e-02, -3.81403230e-02, -4.02087420e-02,
       -3.59698646e-02, -3.31004485e-02, -3.26571278e-02, -3.36864218e-02,
       -3.19018662e-02, -3.40237878e-02, -3.37524377e-02, -3.72081064e-02,
       -3.61542739e-02, -3.64202298e-02, -3.88181508e-02, -4.03619446e-02,
       -4.14696932e-02, -3.99662293e-02, -3.72822434e-02, -3.74110676e-02,
       -3.55924480e-02, -3.57068926e-02, -3.23941596e-02, -3.14334184e-02,
       -2.98618358e-02, -3.01248245e-02, -2.70188786e-02, -2.79250797e-02,
       -2.61004549e-02, -2.67329495e-02, -2.93196104e-02, -2.91609298e-02,
...
        4.53688093e-02,  4.73668501e-02,  4.42357622e-02,  4.33846340e-02,
        3.91795747e-02,  3.76323350e-02,  4.11133021e-02,  4.11174148e-02,
        3.74740437e-02,  3.82420644e-02,  3.57182994e-02,  3.77150252e-02,
        3.54228467e-02,  3.85515019e-02,  3.65166292e-02,  3.74956727e-02,
        3.66724096e-02,  3.90413254e-02,  3.79540958e-02,  3.71193476e-02,
        3.85152549e-02,  3.69921178e-02,  3.74693535e-02,  3.80658731e-02,
        3.64099815e-02,  3.65545265e-02,  3.91732752e-02,  3.88821959e-02,
        4.01163213e-02,  4.34245653e-02,  4.51696441e-02,  4.20900472e-02,
        4.24142443e-02,  4.47618142e-02,  4.65177521e-02,  4.62731756e-02,
        4.72950600e-02,  4.69612703e-02,  4.82777506e-02,  4.56680283e-02,
        4.58845161e-02,  4.48992997e-02,  4.45805192e-02,  4.17784266e-02,
        3.89825553e-02,  3.62436064e-02,  3.79336737e-02,  3.69874202e-02,
        3.71719524e-02,  3.83322574e-02,  3.75876129e-02,  3.83295007e-02,
        3.97606194e-02,  4.12788875e-02,  4.15522866e-02,  3.99039425e-02,
        3.90486643e-02,  4.20593731e-02,  4.09338176e-02,  4.10035066e-02,
        3.95894535e-02,  4.09034379e-02,  4.35777977e-02,  4.45070155e-02,
        4.73690778e-02,  4.76812944e-02,  4.79385667e-02,  5.03006019e-02,
        4.80190329e-02,  4.84119654e-02,  4.88683060e-02,  5.16148619e-02,
        5.36525510e-02,  5.40826619e-02,  5.65216243e-02,  5.49541526e-02,
        5.62267005e-02,  5.23774177e-02,  5.22872321e-02], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### 3 errors: random noise + missing data + orbit error - 3cm

In [113]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_3errors_3cm.glob('*SSHA_gridded*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_3cm/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_3cm/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_3cm/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_3cm/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_3cm/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_3cm/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_3cm/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_3cm/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_3cm/SSHA_gridded_2017-12-25.nc')]


In [114]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 360, time: 923)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [115]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66))
grids

<xarray.Dataset>
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [116]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_3errors=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [117]:
# make new DataArray objects
gmsl_grids_3errors_3cm = xr.DataArray(ssh_weighted_mean_3errors, dims=['time'])
gmsl_grids_3errors_3cm = gmsl_grids_3errors_3cm.assign_coords({'time':grids.time})
gmsl_grids_3errors_3cm.name = 'gmsl_grids'
gmsl_grids_3errors_3cm

<xarray.DataArray 'gmsl_grids' (time: 923)>
array([-3.35377380e-02, -3.18281986e-02, -2.98498403e-02, -3.10477130e-02,
       -3.17227989e-02, -3.39749791e-02, -3.16132940e-02, -3.41284685e-02,
       -3.24078947e-02, -3.99675108e-02, -4.02742103e-02, -4.08394784e-02,
       -4.01046872e-02, -4.30837758e-02, -4.27118987e-02, -3.99593264e-02,
       -4.00506593e-02, -3.77371088e-02, -4.14582044e-02, -3.89013775e-02,
       -3.87437753e-02, -3.79016362e-02, -3.90963480e-02, -3.81005220e-02,
       -3.72613631e-02, -3.81781869e-02, -3.81505899e-02, -3.81844565e-02,
       -3.90212163e-02, -3.70669626e-02, -3.50122638e-02, -3.70116606e-02,
       -3.59822921e-02, -3.32246833e-02, -3.10359299e-02, -3.20651457e-02,
       -2.99796164e-02, -2.75100600e-02, -2.64534429e-02, -2.64409203e-02,
       -3.00844479e-02, -3.11408844e-02, -3.35486233e-02, -3.18751298e-02,
       -2.99212560e-02, -2.75293477e-02, -3.14188153e-02, -3.26906778e-02,
       -3.07232868e-02, -3.41125540e-02, -3.87248136e-02, -4.05781567e-02,
       -3.66516486e-02, -3.27867754e-02, -3.18235122e-02, -3.37723382e-02,
       -3.16829830e-02, -3.35801989e-02, -3.39271054e-02, -3.69837917e-02,
       -3.64768282e-02, -3.56662944e-02, -3.89493965e-02, -4.07478064e-02,
       -4.08534780e-02, -3.98428142e-02, -3.74454781e-02, -3.65923084e-02,
       -3.52493785e-02, -3.64794359e-02, -3.28735672e-02, -3.21556553e-02,
       -3.06408480e-02, -3.02175153e-02, -2.69039981e-02, -2.84927487e-02,
       -2.65220180e-02, -2.72835586e-02, -2.91861575e-02, -2.95521505e-02,
...
        4.58372161e-02,  4.78335693e-02,  4.35561426e-02,  4.27291282e-02,
        3.95774283e-02,  3.73724364e-02,  4.17814702e-02,  4.13328595e-02,
        3.74638885e-02,  3.89026329e-02,  3.70642133e-02,  3.74428630e-02,
        3.53464000e-02,  3.84431519e-02,  3.62560451e-02,  3.71286012e-02,
        3.61555032e-02,  3.88145708e-02,  3.89199480e-02,  3.69432829e-02,
        3.80950794e-02,  3.72213013e-02,  3.78279313e-02,  3.72341089e-02,
        3.66560966e-02,  3.66262533e-02,  3.95315550e-02,  3.98836955e-02,
        4.06662226e-02,  4.26097997e-02,  4.41761129e-02,  4.30002473e-02,
        4.12888080e-02,  4.34986837e-02,  4.72886264e-02,  4.67776395e-02,
        4.81821485e-02,  4.66254391e-02,  4.86878157e-02,  4.68142964e-02,
        4.62535731e-02,  4.51818891e-02,  4.46396172e-02,  4.17818353e-02,
        3.87575589e-02,  3.65387201e-02,  3.74908000e-02,  3.76548357e-02,
        3.78570072e-02,  3.78458276e-02,  3.74321379e-02,  3.86549793e-02,
        4.01273891e-02,  4.07875627e-02,  4.16458063e-02,  3.91725339e-02,
        3.95725556e-02,  4.19352986e-02,  4.10281755e-02,  4.08113971e-02,
        4.00226079e-02,  4.06401232e-02,  4.47782539e-02,  4.45768014e-02,
        4.66133319e-02,  4.80874106e-02,  4.83661257e-02,  5.02499267e-02,
        4.76090014e-02,  4.89525013e-02,  5.01947366e-02,  5.11827730e-02,
        5.35500236e-02,  5.29806465e-02,  5.64717911e-02,  5.41662015e-02,
        5.59313633e-02,  5.17909788e-02,  5.46029471e-02], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### Save to Disk

In [118]:
fname = output_dir / ('GMSL_grids_19922017_noerror_100points.nc')
gmsl_grids.to_netcdf(fname)

In [119]:
fname = output_dir / ('GMSL_grids_19922017_randomnoise_100points.nc')
gmsl_grids_randomnoise.to_netcdf(fname)

In [120]:
fname = output_dir / ('GMSL_grids_19922017_missingdata_100points.nc')
gmsl_grids_missingdata.to_netcdf(fname)

In [121]:
fname = output_dir / ('GMSL_grids_19922017_orbiterror_1cm_100points.nc')
gmsl_grids_orbiterror_1cm.to_netcdf(fname)
fname = output_dir / ('GMSL_grids_19922017_orbiterror_2cm_100points.nc')
gmsl_grids_orbiterror_2cm.to_netcdf(fname)
fname = output_dir / ('GMSL_grids_19922017_orbiterror_3cm_100points.nc')
gmsl_grids_orbiterror_3cm.to_netcdf(fname)

In [122]:
fname = output_dir / ('GMSL_grids_19922017_3errors_1cm_100points.nc')
gmsl_grids_3errors_1cm.to_netcdf(fname)
fname = output_dir / ('GMSL_grids_19922017_3errors_2cm_100points.nc')
gmsl_grids_3errors_2cm.to_netcdf(fname)
fname = output_dir / ('GMSL_grids_19922017_3errors_3cm_100points.nc')
gmsl_grids_3errors_3cm.to_netcdf(fname)

# Compute GMSL from daily ECCO output with simple weighing

### Compute the ocean area, needed to compute GMSL from ECCO

In [4]:
ECCO_grid = xr.open_dataset(str(main_dir) + '/GRID_GEOMETRY_ECCO_V4r4_latlon_0p50deg.nc')
ECCO_grid

<xarray.Dataset>
Dimensions:         (Z: 50, latitude: 360, longitude: 720, nv: 2)
Coordinates:
  * Z               (Z) float32 -5.0 -15.0 -25.0 ... -5.461e+03 -5.906e+03
  * latitude        (latitude) float32 -89.75 -89.25 -88.75 ... 89.25 89.75
  * longitude       (longitude) float32 -179.8 -179.2 -178.8 ... 179.2 179.8
    latitude_bnds   (latitude, nv) float32 ...
    longitude_bnds  (longitude, nv) float32 ...
    Z_bnds          (Z, nv) float32 ...
Dimensions without coordinates: nv
Data variables:
    hFacC           (Z, latitude, longitude) float64 ...
    Depth           (latitude, longitude) float64 ...
    area            (latitude, longitude) float64 ...
    drF             (Z) float32 ...
    maskC           (Z, latitude, longitude) bool ...
Attributes: (12/57)
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Ian Fenty and Ou Wang
    cdm_data_type:                   Grid
    comment:                         Fields provided on a regular lat-lon gri...
    Conventions:                     CF-1.8, ACDD-1.3
    coordinates_comment:             Note: the global 'coordinates' attribute...
    ...                              ...
    references:                      ECCO Consortium, Fukumori, I., Wang, O.,...
    source:                          The ECCO V4r4 state estimate was produce...
    standard_name_vocabulary:        NetCDF Climate and Forecast (CF) Metadat...
    summary:                         This dataset provides geometric paramete...
    title:                           ECCO Geometry Parameters for the 0.5 deg...
    uuid:                            b4795c62-86e5-11eb-9c5f-f8f21e2ee3e0

In [5]:
# mask land points
ECCO_ocean_area_all = ECCO_grid.area*ECCO_grid.maskC[0,:]
ECCO_ocean_area_all=ECCO_ocean_area_all.drop('Z')
ECCO_ocean_area_all

<xarray.DataArray (latitude: 360, longitude: 720)>
array([[       0.        ,        0.        ,        0.        , ...,
               0.        ,        0.        ,        0.        ],
       [       0.        ,        0.        ,        0.        , ...,
               0.        ,        0.        ,        0.        ],
       [       0.        ,        0.        ,        0.        , ...,
               0.        ,        0.        ,        0.        ],
       ...,
       [67431481.94309929, 67431481.94309929, 67431481.94309929, ...,
        67431481.94309929, 67431481.94309929, 67431481.94309929],
       [40460943.34271768, 40460943.34271768, 40460943.34271768, ...,
        40460943.34271768, 40460943.34271768, 40460943.34271768],
       [13487323.48486681, 13487323.48486681, 13487323.48486681, ...,
        13487323.48486681, 13487323.48486681, 13487323.48486681]])
Coordinates:
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8

In [6]:
# mask high latitudes
ECCO_ocean_area=ECCO_ocean_area_all.sel(latitude=slice(-66,66))
ECCO_ocean_area

<xarray.DataArray (latitude: 264, longitude: 720)>
array([[1.26956346e+09, 1.26956346e+09, 1.26956346e+09, ...,
        1.26956346e+09, 1.26956346e+09, 1.26956346e+09],
       [1.29410934e+09, 1.29410934e+09, 1.29410934e+09, ...,
        1.29410934e+09, 1.29410934e+09, 1.29410934e+09],
       [1.31855668e+09, 1.31855668e+09, 1.31855668e+09, ...,
        1.31855668e+09, 1.31855668e+09, 1.31855668e+09],
       ...,
       [1.31855668e+09, 1.31855668e+09, 1.31855668e+09, ...,
        0.00000000e+00, 0.00000000e+00, 1.31855668e+09],
       [0.00000000e+00, 1.29410934e+09, 1.29410934e+09, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])
Coordinates:
  * latitude   (latitude) float32 -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8

In [23]:
ECCO_total_ocean_area_all = ECCO_ocean_area_all.sum(skipna=True).values
ECCO_total_ocean_area_all

array(3.5800207e+14)

In [24]:
ECCO_total_ocean_area_zonal_all = ECCO_ocean_area_all.sum(dim=['longitude'],skipna=True).values
ECCO_total_ocean_area_zonal_all.shape

(360,)

In [25]:
ECCO_total_ocean_area = ECCO_ocean_area.sum(skipna=True).values
ECCO_total_ocean_area

array(3.36736828e+14)

In [26]:
ECCO_total_ocean_area_zonal = ECCO_ocean_area.sum(dim=['longitude'],skipna=True).values
ECCO_total_ocean_area_zonal.shape

(264,)

### Get the list of ECCO daily data in the cloud

In [3]:
# %%capture
# import requests

# def store_aws_keys(endpoint: str="https://archive.podaac.earthdata.nasa.gov/s3credentials"):    
#     with requests.get(endpoint, "w") as r:
#         accessKeyId, secretAccessKey, sessionToken, expiration = list(r.json().values())

#     creds ={}
#     creds['AccessKeyId'] = accessKeyId
#     creds['SecretAccessKey'] = secretAccessKey
#     creds['SessionToken'] = sessionToken
#     creds['expiration'] = expiration
    
#     return creds

# creds = store_aws_keys()
# print(creds)

In [4]:
# ShortName = "ECCO_L4_SSH_05DEG_DAILY_V4R4B"
# # Ask PODAAC for the collection id
# response = requests.get(
#     url='https://cmr.earthdata.nasa.gov/search/collections.umm_json', 
#     params={'provider': "POCLOUD",
#             'ShortName': ShortName,
#             'page_size': 1}
# )
# ummc = response.json()['items']
# # ummc
# # used to work, doesn't work anymore...
# # ummc = response.json()['items'][0]
# # ccid = ummc['meta']['concept-id']
# # print(f'collection id: {ccid}')

In [5]:
# s3 = s3fs.S3FileSystem(
#     key=creds['AccessKeyId'],
#     secret=creds['SecretAccessKey'],
#     token=creds['SessionToken'],
#     client_kwargs={'region_name':'us-west-2'},
# )

In [6]:
# # make a S3 'filesystem' object
# fs = s3fs.S3FileSystem(anon=False,
#                       key=creds['AccessKeyId'],
#                       secret=creds['SecretAccessKey'],
#                       token=creds['SessionToken'])

### Read ECCO data from the cloud

In [7]:
# ECCO_SSH_files = fs.glob(join("podaac-ops-cumulus-protected/", ShortName, '*.nc'))
# # ECCO_SSH_files

In [8]:
# #we only start on 09-23-1992, doy=266
# ECCO_SSH_files=ECCO_SSH_files[266:]
# # ECCO_SSH_files

In [9]:
# #Load all of the files from AWS S3 using 'direct connection' and combine into a single xarray DataSet object
# paths=[fs.open(f) for f in ECCO_SSH_files] 
# ECCO_DS_daily_all = xr.open_mfdataset(
#     paths=paths,
#     combine='by_coords',
#     # concat_dim='time',
#     decode_cf=True,
#     coords='minimal',
#     chunks={'time': 1}  
# )
# ECCO_DS_daily_all.close()
# ECCO_DS_daily_all

<xarray.Dataset>
Dimensions:         (time: 9231, latitude: 360, longitude: 720, nv: 2)
Coordinates:
  * time            (time) datetime64[ns] 1992-09-23T12:00:00 ... 2017-12-31T...
  * latitude        (latitude) float32 -89.75 -89.25 -88.75 ... 89.25 89.75
  * longitude       (longitude) float32 -179.8 -179.2 -178.8 ... 179.2 179.8
    time_bnds       (time, nv) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    latitude_bnds   (latitude, nv) float32 dask.array<chunksize=(360, 2), meta=np.ndarray>
    longitude_bnds  (longitude, nv) float32 dask.array<chunksize=(720, 2), meta=np.ndarray>
Dimensions without coordinates: nv
Data variables:
    SSH             (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    SSHIBC          (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    SSHNOIBC        (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes: (12/57)
    acknowledgement:              This research was carried out by the Jet Pr...
    author:                       Ou Wang and Ian Fenty
    cdm_data_type:                Grid
    comment:                      Fields provided on a regular lat-lon grid. ...
    Conventions:                  CF-1.8, ACDD-1.3
    coordinates_comment:          Note: the global 'coordinates' attribute de...
    ...                           ...
    time_coverage_duration:       P1D
    time_coverage_end:            1992-09-24T00:00:00
    time_coverage_resolution:     P1D
    time_coverage_start:          1992-09-23T00:00:00
    title:                        ECCO Sea Surface Height - Daily Mean 0.5 De...
    uuid:                         44a9c25e-9a8a-414c-85e0-ebe688de113d

In [10]:
# ECCO_SSH_all = ECCO_DS_daily_all.SSH-ecco_mean_all.mean_ssh

In [11]:
# ECCO_SSH_all = ECCO_SSH_all.resample(time="10D",loffset=datetime.timedelta(days=4)).mean()

In [12]:
# fname = output_dir / ('ECCO_SSH_19922017_10days_nosicapplied.nc')
# ECCO_SSH_all.to_netcdf(fname)

In [13]:
# # Load SIC files
# ECCO_SIC_files = list(sic_dir.glob('*.nc'))
# ECCO_SIC_files=np.sort(ECCO_SIC_files)
# ECCO_SIC_daily = xr.open_mfdataset(
#     paths=ECCO_SIC_files,
#     combine='nested',
#     concat_dim='time',
#     compat='override',
#     decode_cf=True,
#     coords='minimal',
#     chunks={'time': 1}  
# )

In [14]:
# #mask sea ice
# ECCO_DS_daily_all=ECCO_DS_daily_all.where(ECCO_SIC_daily.SIarea <= 0) #change to NaN everything that is False
# del ECCO_SIC_files
# gc.collect()
# del ECCO_SIC_daily
# gc.collect()

0

In [15]:
# ECCO_SSH_all = ECCO_DS_daily_all.SSH-ecco_mean_all.mean_ssh
# del ECCO_DS_daily_all
# gc.collect()

34

In [16]:
# ECCO_SSH_all=ECCO_SSH_all.resample(time="10D",loffset=datetime.timedelta(days=4)).mean()

In [17]:
# fname = output_dir / ('ECCO_SSH_19922017_10days_sicapplied.nc')
# ECCO_SSH_all.to_netcdf(fname)

In [3]:
ECCO_SSH_all=xr.open_dataarray(output_dir / ('ECCO_SSH_19922017_sicapplied.nc'))
ECCO_SSH_all.data

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan

In [5]:
ECCO_SSH=ECCO_SSH_all.sel(latitude=slice(-66,66))
ECCO_SSH

<xarray.DataArray (time: 924, latitude: 264, longitude: 720)>
[175633920 values with dtype=float32]
Coordinates:
  * latitude   (latitude) float32 -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### Calculate the True daily GMSL from ECCO

In [22]:
#!!!!!PROBLEM WITH SEA ICE NOT MASKED IN AREA COMPUTATION WITH THIS METHOD !!!!
# first call sets up the calculation in dask
# ... \sum_i [SSH_i x grid cell area_i] / total grid cell area
ECCO_global_mean_sea_level_old = (ECCO_SSH * ECCO_ocean_area).sum(dim=['latitude','longitude'],skipna=True)/ECCO_total_ocean_area

# second call actually computes it
ECCO_global_mean_sea_level_old = ECCO_global_mean_sea_level_old.compute()

In [24]:
ECCO_global_mean_sea_level_all_old.name = 'gmsl_ecco'
fname = output_dir / ('GMSL_ECCO_dailyfiles_19922017_alllat_old.nc')
ECCO_global_mean_sea_level_all_old.to_netcdf(fname)

In [23]:
# first call sets up the calculation in dask
# ... \sum_i [SSH_i x grid cell area_i] / total grid cell area
ECCO_global_mean_sea_level_all_old = (ECCO_SSH_all * ECCO_ocean_area_all).sum(dim=['latitude','longitude'],skipna=True)/ECCO_total_ocean_area_all

# second call actually computes it
ECCO_global_mean_sea_level_all_old = ECCO_global_mean_sea_level_all_old.compute()

In [24]:
ECCO_global_mean_sea_level_old.name = 'gmsl_ecco'
fname = output_dir / ('GMSL_ECCO_dailyfiles_19922017_old.nc')
ECCO_global_mean_sea_level_old.to_netcdf(fname)

In [12]:
[LO,LA]=np.meshgrid(ECCO_SSH.longitude,ECCO_SSH.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [ECCO_SSH.time.shape[0], 1, 1])
weights[np.where(~np.isfinite(ECCO_SSH.data))]=np.nan
ECCO_global_mean_sea_level_tmp=np.nansum(ECCO_SSH*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [13]:
ECCO_global_mean_sea_level = xr.Dataset({"gmsl_ecco": (("time"), ECCO_global_mean_sea_level_tmp)},
            coords={"time": ECCO_SSH.time})
del ECCO_global_mean_sea_level_tmp
ECCO_global_mean_sea_level['gmsl_ecco'].attrs['unit'] = 'm'
fname = output_dir / ('GMSL_ECCO_dailyfiles_19922017.nc')
ECCO_global_mean_sea_level.to_netcdf(fname)
ECCO_global_mean_sea_level

<xarray.Dataset>
Dimensions:    (time: 924)
Coordinates:
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    gmsl_ecco  (time) float32 -0.03441 -0.03181 -0.03035 ... 0.05291 0.05486

In [4]:
[LO,LA]=np.meshgrid(ECCO_SSH_all.longitude,ECCO_SSH_all.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [ECCO_SSH_all.time.shape[0], 1, 1])

In [5]:
ind=np.where(~np.isfinite(ECCO_SSH_all.data))
ind

(array([  0,   0,   0, ..., 923, 923, 923]),
 array([  0,   0,   0, ..., 359, 359, 359]),
 array([  0,   1,   2, ..., 717, 718, 719]))

In [6]:
weights[ind]

array([0.00436336, 0.00436336, 0.00436336, ..., 0.00436336, 0.00436336,
       0.00436336], dtype=float32)

In [7]:
weights[np.where(~np.isfinite(ECCO_SSH_all.data))]=np.nan
weights[ind]

array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)

In [8]:
ECCO_global_mean_sea_level_all_tmp=np.nansum(ECCO_SSH_all*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))
del weights

In [11]:
ECCO_global_mean_sea_level_all = xr.Dataset({"gmsl_ecco": (("time"), ECCO_global_mean_sea_level_all_tmp)},
            coords={"time": ECCO_SSH_all.time})
del ECCO_global_mean_sea_level_all_tmp
ECCO_global_mean_sea_level_all['gmsl_ecco'].attrs['unit'] = 'm'
fname = output_dir / ('GMSL_ECCO_dailyfiles_19922017_alllat.nc')
ECCO_global_mean_sea_level_all.to_netcdf(fname)
ECCO_global_mean_sea_level_all

<xarray.Dataset>
Dimensions:    (time: 924)
Coordinates:
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    gmsl_ecco  (time) float32 -0.03485 -0.03264 -0.031 ... 0.05292 0.05436

### Zonal plots ECCO GMSL

In [ ]:
ECCO_global_mean_sea_level_zonal_all = (ECCO_SSH_all * ECCO_ocean_area_all).sum(dim=['longitude'],skipna=True)/ECCO_total_ocean_area_zonal_all
ECCO_global_mean_sea_level_zonal_all = ECCO_global_mean_sea_level_zonal_all.compute()
ECCO_global_mean_sea_level_zonal_all
ECCO_global_mean_sea_level_zonal = (ECCO_SSH * ECCO_ocean_area).sum(dim=['longitude'],skipna=True)/ECCO_total_ocean_area_zonal
ECCO_global_mean_sea_level_zonal = ECCO_global_mean_sea_level_zonal.compute()

In [ ]:
(ECCO_global_mean_sea_level_zonal_all).plot();
plt.grid()
plt.ylabel('zonal GMSL (m)')

In [ ]:
(ECCO_global_mean_sea_level_zonal).plot();
plt.grid()
plt.ylabel('zonal GMSL (m)')

In [ ]:
ECCO_global_slope_mean_sea_level_zonal=ECCO_global_mean_sea_level_zonal.polyfit(dim='time', deg=1, skipna=True, rcond=None, w=None, full=False, cov=False)
ECCO_global_slope_mean_sea_level_zonal

In [ ]:
(ECCO_global_slope_mean_sea_level_zonal.polyfit_coefficients[0,:]).plot();
plt.grid()
plt.ylabel('GMSL slope (m)')
plt.xlabel('Latitude')
# fname = output_dir / ('temp.nc')
# ECCO_global_mean_sea_level_zonal.to_netcdf(fname)

### Variance ECCO GMSL

In [ ]:
(ECCO_SSH.var(dim=('latitude','longitude'),skipna=True)).plot(label='+-66');
(ECCO_SSH_all.var(dim=('latitude','longitude'),skipna=True)).plot(label='+-90');
plt.grid()
plt.ylabel('var GMSL (m)')
plt.legend(ncol=2, loc='lower right',fontsize=8)

In [ ]:
((ECCO_SSH.var(dim=('latitude','longitude'),skipna=True))-(ECCO_SSH.var(dim=('latitude','longitude'),skipna=True)).mean()).plot(label='+-66');
((ECCO_SSH_all.var(dim=('latitude','longitude'),skipna=True))-(ECCO_SSH_all.var(dim=('latitude','longitude'),skipna=True)).mean()).plot(label='+-90');
plt.grid()
plt.ylabel('var GMSL - mean (m)')
plt.legend(ncol=2, loc='lower right',fontsize=8)

# Compute GMSL from Measures grids

In [ ]:
JPL_grids_files = list(grids_file_dir_JPL.glob('*ssh_grids_v2205_*nc'))
JPL_grids_files=np.sort(JPL_grids_files)
print(JPL_grids_files[0:5])
print('...')
print(JPL_grids_files[-5:-1])

In [ ]:
# import os
# for i in range(0,len(JPL_grids_files)):
#     try:
#         grids = xr.open_dataset(JPL_grids_files[i])
#         ind=np.where(~np.isfinite(grids.SLA))
#     except:
#         print(JPL_grids_files[i])
#         os.remove(JPL_grids_files[i])

In [ ]:
grids = xr.open_mfdataset(
        paths=JPL_grids_files,
        combine='nested',
        concat_dim='Time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   

In [36]:
grids=grids.sel(Latitude=slice(-66,66))
grids

In [37]:
[LO,LA]=np.meshgrid(grids.Longitude,grids.Latitude)
weights = np.cos(np.deg2rad(LA))

In [38]:
weights=np.tile(weights, [grids.SLA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SLA))]=np.nan
weights.shape

(2190, 792, 2160)

In [39]:
ssh_weighted_mean=np.nansum(grids.SLA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [40]:
# make new DataArray objects
gmsl_grids_jpl = xr.DataArray(ssh_weighted_mean, dims=['time'])
gmsl_grids_jpl = gmsl_grids_jpl.assign_coords({'time':grids.Time.data})
gmsl_grids_jpl.name = 'gmsl_jpl_grids'
gmsl_grids_jpl

<xarray.DataArray 'gmsl_jpl_grids' (time: 2190)>
array([-0.00232378, -0.0038849 , -0.0053242 , ...,  0.07769491,
        0.07858239,  0.07980166], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-10-10T12:00:00 ... 2022-09-28T12:00:00

In [41]:
fname = output_dir / ('GMSL_jplgrids_19922022_jpl.nc')
gmsl_grids_jpl.to_netcdf(fname)

# Compute GMSL from simple gridder of Beckley along track data daily files

In [17]:
beckley_grids_files = list(grids_file_dir_beckley.glob('ssha_global_half_deg_*nc'))
beckley_grids_files=np.sort(beckley_grids_files)
print(beckley_grids_files[0:5])
print('...')
print(beckley_grids_files[-5:-1])

[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19920927.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921007.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921017.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921027.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921106.nc')]
...
[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171115.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171125.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171205.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171215.nc')]


In [18]:
grids = xr.open_mfdataset(
        paths=beckley_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   

In [19]:
grids=grids.sel(latitude=slice(-66,66))
grids

In [20]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))

In [21]:
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
weights.shape

(923, 264, 720)

In [22]:
ssh_weighted_mean=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [23]:
# make new DataArray objects
gmsl_grids_beckley = xr.DataArray(ssh_weighted_mean, dims=['time'])
gmsl_grids_beckley = gmsl_grids_beckley.assign_coords({'time':grids.time.data})
gmsl_grids_beckley.name = 'gmsl_grids_beckley'
gmsl_grids_beckley

<xarray.DataArray 'gmsl_grids_beckley' (time: 923)>
array([ 1.69078968e-02,  1.22925006e-02,  5.12305181e-03, -2.19792710e-03,
       -4.06668615e-03, -6.64573582e-03, -2.11527734e-03, -1.14462320e-02,
       -1.45043293e-02, -1.74436271e-02, -1.88453346e-02, -2.26884615e-02,
       -2.23314259e-02, -2.40905453e-02, -2.02088933e-02, -1.89829972e-02,
       -1.99991670e-02, -1.91079061e-02, -1.94311906e-02, -2.31938921e-02,
       -2.08824035e-02, -1.86882280e-02, -2.18924358e-02, -2.16961987e-02,
       -2.36001033e-02, -2.31348369e-02, -2.14335956e-02, -2.03797575e-02,
       -1.93761177e-02, -1.89929288e-02, -1.43261943e-02, -1.81000289e-02,
       -1.63723864e-02, -1.35198329e-02, -1.31252529e-02, -1.28582120e-02,
       -1.41602689e-02, -9.68575291e-03, -6.68572308e-03, -8.80166516e-03,
       -1.29868602e-02, -1.21212536e-02, -1.35079222e-02, -1.48071954e-02,
       -1.17592970e-02, -9.72176902e-03, -1.14215296e-02, -1.41173787e-02,
       -1.05190007e-02, -1.19873947e-02, -1.90570801e-02, -2.01623477e-02,
       -1.82843711e-02, -1.60147399e-02, -1.48031795e-02, -1.39125763e-02,
       -1.45940073e-02, -1.59441214e-02, -1.74911264e-02, -1.89811327e-02,
       -1.92872398e-02, -1.95422471e-02, -1.88861806e-02, -1.84019376e-02,
       -2.42679678e-02, -1.77722406e-02, -1.56562086e-02, -1.65406577e-02,
       -1.39443502e-02, -1.33155202e-02, -1.09935226e-02, -1.00437356e-02,
       -7.76944822e-03, -8.11815914e-03, -5.58401877e-03, -5.82845928e-03,
       -4.42570262e-03, -2.48672115e-03, -5.35771344e-03, -9.86848027e-03,
...
        6.17260821e-02,  5.88918738e-02,  6.22275062e-02,  6.12512603e-02,
        5.88533916e-02,  5.69612235e-02,  5.29930666e-02,  5.76765947e-02,
        5.95689416e-02,  5.55902310e-02,  5.77683114e-02,  5.58906086e-02,
        5.58774211e-02,  5.30412532e-02,  5.64005896e-02,  5.10226078e-02,
        5.19252084e-02,  5.03525101e-02,  4.94241379e-02,  4.87880781e-02,
        4.76776771e-02,  5.11273891e-02,  5.19718938e-02,  5.30977473e-02,
        5.29277176e-02,  5.31908311e-02,  5.14503159e-02,  5.21013886e-02,
        5.23775406e-02,  5.35147823e-02,  5.97303249e-02,  6.17798716e-02,
        6.10412098e-02,  5.81164286e-02,  6.01729043e-02,  6.31641522e-02,
        6.32677749e-02,  6.41530827e-02,  6.50055856e-02,  6.46272078e-02,
        6.00098744e-02,  5.96902855e-02,  5.74641638e-02,  5.69302924e-02,
        5.59988245e-02,  5.36370538e-02,  5.10248765e-02,  5.27026169e-02,
        5.25101796e-02,  5.31482399e-02,  5.41420504e-02,  5.27642220e-02,
        5.18646277e-02,  5.22756465e-02,  5.01393415e-02,  4.87993285e-02,
        4.85423058e-02,  4.89996560e-02,  5.02156578e-02,  5.23073487e-02,
        5.20371869e-02,  5.28719090e-02,  5.16298823e-02,  5.63817807e-02,
        5.80990314e-02,  5.99170849e-02,  6.02247044e-02,  6.28706142e-02,
        6.41483217e-02,  6.29180223e-02,  6.42790198e-02,  6.48969039e-02,
        6.75930232e-02,  6.57676682e-02,  6.73445165e-02,  6.72886148e-02,
        6.67184889e-02,  6.61361068e-02,  6.26157001e-02], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2017-12-25

In [24]:
fname = output_dir / ('GMSL_Beckley_dailyfiles_grids.nc')
gmsl_grids_beckley.to_netcdf(fname)